# Results

In [1]:
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource
from bokeh.layouts import row
from bokeh.palettes import Spectral6


import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select"

output_notebook();

Loading BokehJS ...

## chifflot-8.lille.grid5000.fr_1
### Performace -CPU

In [2]:
data = json.load(open('results/chifflot-8.lille.grid5000.fr_1/results_cpu.json', 'r'))
labels =  [key for key, value in data.items()]
i = 0

colors = ['green', 'red', 'blue', 'darkturquoise', 'coral']

p1 = figure(title="Execution Times - CPU", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p2 = figure(title="Error - CPU", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'Matrix size (N x N)'
p1.yaxis.axis_label = 'Time ( ms )'
p2.xaxis.axis_label = 'Matrix size (N x N)'
p2.yaxis.axis_label = 'MSE'

errors_total = list()

for label in labels:
    times = list()
    errors = list()
    data_err = data[label]
    sizes = [key for key, value in data_err.items()]
    for size in sizes:
        times.append(data_err[size]['time'])
        errors.append(data_err[size]['error'])
    sizes = list(map(int, sizes))

    data_plot = pd.DataFrame({'size':sizes, 'time':times, 'error':errors})
    # Add error bars to the DataFrame
    errors_total += [errors]
    errors = list(map(float, errors))
    errors_t = dict()
    errors_t['sizes'] = sizes
    for key, value in zip(labels, errors_total):
        errors_t[key] = value
    source_cpu_errors = ColumnDataSource(pd.DataFrame(errors_t))

    
    data_plot['error_low'] = data_plot['time'] - np.array(errors)
    data_plot['error_high'] = data_plot['time'] + np.array(errors)
    data_plot = data_plot.sort_values(by=['size'], ascending = True)
    data_plot = data_plot.reset_index(drop=True)
    source_cpu = ColumnDataSource(data_plot)
    
    p1.square(source=source_cpu, x='size', y='time', legend_label=labels[i], fill_color=None, line_color=colors[i])
    p1.line(source=source_cpu, x='size', y='time', line_color=colors[i])
    
    if label != 'serial':
        #p2.line(source=source_cpu_errors, x='sizes', y=label, legend_label=labels[i], color=colors[i])
        p2.square(source=source_cpu_errors, x='sizes', y=label, legend_label=labels[i], color=colors[i])
    
    if label == 'mkl':
        data_mkl_chifflot = data_plot.copy(deep = True)
        error_mkl_chifflot = pd.DataFrame(errors_t)
        
    if label == 'openMP':
        data_openMP_chifflot = data_plot.copy(deep = True)
        error_openMP_chifflot = pd.DataFrame(errors_t)
        
    i=i+1

maxi = max(max([sublist[:] for sublist in errors_total]))


p2.y_range.end = maxi*1.1

p1.legend.location = "top_left"
p2.legend.location = "top_left"

show(gridplot([p1, p2], ncols=2))

In [3]:
p1 = figure(title="Execution Times - CPU", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p2 = figure(title="Error - CPU", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'Matrix size (N x N)'
p1.yaxis.axis_label = 'Time ( ms )'

p2.xaxis.axis_label = 'Matrix size (N x N)'
p2.yaxis.axis_label = 'MSE'

source_mkl_chifflot = ColumnDataSource(data_mkl_chifflot)
source_mkl_errors_chifflot = ColumnDataSource(error_mkl_chifflot)
source_openmp_chifflot = ColumnDataSource(data_openMP_chifflot)
source_openmp_errors_chifflot = ColumnDataSource(error_openMP_chifflot)

p1.square(source=source_mkl_chifflot, x='size', y='time', legend_label='mkl', fill_color=None, line_color='blue')
p1.line(source=source_mkl_chifflot, x='size', y='time', line_color='blue')
p1.square(source=source_openmp_chifflot, x='size', y='time', legend_label='openMP', fill_color=None, line_color='darkturquoise')
p1.line(source=source_openmp_chifflot, x='size', y='time', line_color='darkturquoise')

p1.legend.location = "top_left"

p2.square(source=source_mkl_errors_chifflot, x='sizes', y='mkl', legend_label='mkl', color='blue', fill_alpha=0.4)
#p2.line(source=source_mkl_errors, x='sizes', y='mkl', line_color='blue')
p2.square(source=source_openmp_errors_chifflot, x='sizes', y='openMP', legend_label='openMP', color='darkturquoise', fill_alpha=0.4)
#p2.line(source=source_openmp_errors, x='sizes', y='openMP', line_color='darkturquoise')
p2.y_range.end = maxi*1.1
p2.legend.location = "top_left"

show(gridplot([p1, p2], ncols=2))

### Performace -GPU

In [4]:
data = json.load(open('results/chifflot-8.lille.grid5000.fr_1/results_gpu.json', 'r'))
labels =  [key for key, value in data.items()]
i = 0

colors = ['mediumseagreen', 'tomato', 'dodgerblue', 'deeppink', 'darkviolet']
p1 = figure(title="Execution Times - GPU", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p2 = figure(title="Error - GPU", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'Matrix size (N x N)'
p1.yaxis.axis_label = 'Time ( ms )'

p2.xaxis.axis_label = 'Matrix size (N x N)'
p2.yaxis.axis_label = 'MSE'

errors_total = list()

for label in labels:
    times = list()
    errors = list()
    data_err = data[label]
    sizes = [key for key, value in data_err.items()]
    for size in sizes:
        times.append(data_err[size]['time'])
        errors.append(data_err[size]['error'])
    sizes = list(map(int, sizes))

    data_plot = pd.DataFrame({'size':sizes, 'time':times, 'error':errors})
    # Add error bars to the DataFrame
    errors_total += [errors]
    errors = list(map(float, errors))
    errors_t = dict()
    errors_t['sizes'] = sizes
    for key, value in zip(labels, errors_total):
        errors_t[key] = value
    source_gpu_errors = ColumnDataSource(pd.DataFrame(errors_t))

    
    data_plot['error_low'] = data_plot['time'] - np.array(errors)
    data_plot['error_high'] = data_plot['time'] + np.array(errors)
    data_plot = data_plot.sort_values(by=['size'], ascending = True)
    data_plot = data_plot.reset_index(drop=True)
    source_gpu = ColumnDataSource(data_plot)
    
    p1.square(source=source_gpu, x='size', y='time', legend_label=labels[i], fill_color=None, line_color=colors[i])
    p1.line(source=source_gpu, x='size', y='time', line_color=colors[i])
    
    if label != 'serial':
        p2.square(source=source_gpu_errors, x='sizes', y=label, legend_label=labels[i], color=colors[i])
        #p2.line(source=source_gpu_errors, x='sizes', y=label, legend_label=labels[i], color=colors[i])
    
    if label == 'cuBLAS_Tensor':
        data_cublas_tensor_chifflot = data_plot.copy(deep = True)
        error_cublas_tensor_chifflot = pd.DataFrame(errors_t)
    if label == 'cuBLAS':
        data_cublas_chifflot = data_plot.copy(deep = True)
        error_cublas_chifflot = pd.DataFrame(errors_t)
        
    i=i+1

maxi = max(max([sublist[:] for sublist in errors_total]))


p2.y_range.end = maxi*1.1

p1.legend.location = "top_left"
p2.legend.location = "top_left"

show(gridplot([p1, p2], ncols=2))

In [5]:
p1 = figure(title="Execution Times - GPU", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p2 = figure(title="Error - GPU", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'Matrix size (N x N)'
p1.yaxis.axis_label = 'Time ( ms )'

p2.xaxis.axis_label = 'Matrix size (N x N)'
p2.yaxis.axis_label = 'MSE'


colors = ['green', 'red', 'blue', 'black', 'coral']

source_cublasT_chifflot = ColumnDataSource(data_cublas_tensor_chifflot)
source_cublasT_errors_chifflot = ColumnDataSource(error_cublas_tensor_chifflot)
source_cublas_chifflot = ColumnDataSource(data_cublas_chifflot)
source_cublas_errors_chifflot = ColumnDataSource(error_cublas_chifflot)

p1.square(source=source_cublasT_chifflot, x='size', y='time', legend_label='cuBLAS_Tensor', fill_color=None, line_color='tomato')
p1.line(source=source_cublasT_chifflot, x='size', y='time', line_color='tomato')
p1.square(source=source_cublas_chifflot, x='size', y='time', legend_label='cuBLAS', fill_color=None, line_color='mediumseagreen')
p1.line(source=source_cublas_chifflot, x='size', y='time', line_color='mediumseagreen')

p1.legend.location = "top_left"

p2.square(source=source_cublasT_errors_chifflot, x='sizes', y='cuBLAS_Tensor', legend_label='cuBLAS_Tensor', color='tomato', fill_alpha=0.4)
#p2.line(source=source_cublasT_errors, x='sizes', y='cuBLAS_Tensor', line_color='tomato')
p2.square(source=source_cublas_errors_chifflot, x='sizes', y='cuBLAS', legend_label='cuBLAS', color='mediumseagreen', fill_alpha=0.4)
#p2.line(source=source_cublas_errors, x='sizes', y='cuBLAS', line_color='mediumseagreen')
p2.y_range.end = maxi*1.1
p2.legend.location = "top_left"



show(gridplot([p1, p2], ncols=2))

### Performace - SYCL/CPU SYCL/GPU

In [6]:
data1 = json.load(open('results/chifflot-8.lille.grid5000.fr_1/results_sycl_cpu.json', 'r'))
labels =  [key for key, value in data1.items()]
i = 0

colors = ['green', 'red', 'blue', 'black', 'coral']
p1 = figure(title="Execution Times - SYCL/CPU Vs. MKL", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p1.xaxis.axis_label = 'Matrix size (N x N)'
p1.yaxis.axis_label = 'Time ( ms )'


for label in labels:
    times = list()
    errors = list()
    data_err = data1[label]
    sizes = [key for key, value in data_err.items()]
    for size in sizes:
        times.append(data_err[size]['time'])
        errors.append(data_err[size]['error'])
    sizes = list(map(int, sizes))

    data_plot = pd.DataFrame({'size':sizes, 'time':times, 'error':errors})
    # Add error bars to the DataFrame
    errors = list(map(float, errors))
    data_plot['error_low'] = data_plot['time'] - np.array(errors)
    data_plot['error_high'] = data_plot['time'] + np.array(errors)
    data_plot = data_plot.sort_values(by=['size'], ascending = True)
    data_plot = data_plot.reset_index(drop=True)
    source_sycl_chifflot = ColumnDataSource(data_plot)
    source_mkl_chifflot = ColumnDataSource(data_mkl_chifflot)
    
    p1.square(source=source_sycl_chifflot, x='size', y='time', legend_label=labels[i], fill_color=None, line_color='darkturquoise')
    p1.line(source=source_sycl_chifflot, x='size', y='time', line_color='darkturquoise')
    
    p1.square(source=source_mkl_chifflot, x='size', y='time', legend_label='mkl', fill_color=None, line_color='blue')
    p1.line(source=source_mkl_chifflot, x='size', y='time', line_color='blue')
    
    i=i+1

#--------------------------------------------------------------------------------------------------------------------

data2 = json.load(open('results/chifflot-8.lille.grid5000.fr_1/results_sycl_gpu.json', 'r'))
labels =  [key for key, value in data2.items()]
i = 0

p2 = figure(title="Execution Times - SYCL/GPU Vs. cuBLAS", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p2.xaxis.axis_label = 'Matrix size (N x N)'
p2.yaxis.axis_label = 'Time ( ms )'

for label in labels:
    times = list()
    errors = list()
    data_err = data2[label]
    sizes = [key for key, value in data_err.items()]
    for size in sizes:
        times.append(data_err[size]['time'])
        errors.append(data_err[size]['error'])
    sizes = list(map(int, sizes))

    data_plot = pd.DataFrame({'size':sizes, 'time':times, 'error':errors})
    # Add error bars to the DataFrame
    errors = list(map(float, errors))
    data_plot['error_low'] = data_plot['time'] - np.array(errors)
    data_plot['error_high'] = data_plot['time'] + np.array(errors)
    data_plot = data_plot.sort_values(by=['size'], ascending = True)
    data_plot = data_plot.reset_index(drop=True)
    
    source_sycl_gpu_chifflot = ColumnDataSource(data_plot)
    source_cublasT_chifflot = ColumnDataSource(data_cublas_tensor_chifflot)
    source_cublas_chifflot = ColumnDataSource(data_cublas_chifflot)
    
    p2.square(source=source_sycl_gpu_chifflot, x='size', y='time', legend_label=labels[i], fill_color=None, line_color='mediumpurple')
    p2.line(source=source_sycl_gpu_chifflot, x='size', y='time', line_color='mediumpurple')
    p2.segment(source=source_sycl_gpu_chifflot, x0='size', y0='error_low', x1='size', y1='error_high', line_width=2)
    
    p2.square(source=source_cublasT_chifflot, x='size', y='time', legend_label='cuBLAS_Tensor', fill_color=None, line_color='tomato')
    p2.line(source=source_cublasT_chifflot, x='size', y='time', line_color='tomato')
    p2.segment(source=source_cublasT_chifflot, x0='size', y0='error_low', x1='size', y1='error_high', line_width=2)
    
    p2.square(source=source_cublas_chifflot, x='size', y='time', legend_label='cuBLAS', fill_color=None, line_color='mediumseagreen')
    p2.line(source=source_cublas_chifflot, x='size', y='time', line_color='mediumseagreen')
    p2.segment(source=source_cublas_chifflot, x0='size', y0='error_low', x1='size', y1='error_high', line_width=2)

    i=i+1
    
p1.legend.location = "top_left"  
p2.legend.location = "top_left"

show(gridplot([p1, p2], ncols=2))

### Performace - mkl/cuBLAS

In [7]:
p1 = figure(title="Execution Times - MKL Vs. cuBLAS", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p2 = figure(title="Error MKL Vs. cuBLAS", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p3 = figure(title="Execution Times - MKL Vs. cuBLAS Tensor", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p4 = figure(title="Error MKL Vs. cuBLAS Tensor", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")


p1.xaxis.axis_label = 'Matrix size (N x N)'
p1.yaxis.axis_label = 'Time ( ms )'

p2.xaxis.axis_label = 'Matrix size (N x N)'
p2.yaxis.axis_label = 'MSE'

p3.xaxis.axis_label = 'Matrix size (N x N)'
p3.yaxis.axis_label = 'Time ( ms )'

p4.xaxis.axis_label = 'Matrix size (N x N)'
p4.yaxis.axis_label = 'MSE'


source_cublasT_chifflot = ColumnDataSource(data_cublas_tensor_chifflot)
source_mkl_chifflot = ColumnDataSource(data_mkl_chifflot)
source_cublas_chifflot = ColumnDataSource(data_cublas_chifflot)

source_cublasT_errors_chifflot = ColumnDataSource(error_cublas_tensor_chifflot)
source_cublas_errors_chifflot = ColumnDataSource(error_cublas_chifflot)
source_mkl_errors_chifflot = ColumnDataSource(error_mkl_chifflot)


p1.square(source=source_cublas_chifflot, x='size', y='time', legend_label='cuBLAS', fill_color=None, line_color='mediumseagreen')
p1.line(source=source_cublas_chifflot, x='size', y='time', line_color='mediumseagreen')
p1.square(source=source_mkl_chifflot, x='size', y='time', legend_label='mkl', fill_color=None, line_color='blue')
p1.line(source=source_mkl_chifflot, x='size', y='time', line_color='blue')

p2.square(source=source_cublas_errors_chifflot, x='sizes', y='cuBLAS', legend_label='cuBLAS', color='mediumseagreen', fill_alpha=0.4)
#p2.line(source=source_cublas_errors, x='sizes', y='cuBLAS', line_color='mediumseagreen')
p2.square(source=source_mkl_errors_chifflot, x='sizes', y='mkl', legend_label='mkl', color='blue', fill_alpha=0.4)
#p2.line(source=source_mkl_errors, x='sizes', y='mkl', line_color='blue')

p3.square(source=source_cublasT_chifflot, x='size', y='time', legend_label='cuBLAS_Tensor', fill_color=None, line_color='tomato')
p3.line(source=source_cublasT_chifflot, x='size', y='time', line_color='tomato')
p3.square(source=source_mkl_chifflot, x='size', y='time', legend_label='mkl', fill_color=None, line_color='blue')
p3.line(source=source_mkl_chifflot, x='size', y='time', line_color='blue')

p4.square(source=source_cublasT_errors_chifflot, x='sizes', y='cuBLAS_Tensor', legend_label='cuBLAS_Tensor', color='tomato', fill_alpha=0.4)
#p4.line(source=source_cublasT_errors, x='sizes', y='cuBLAS_Tensor', line_color='tomato')
p4.square(source=source_mkl_errors_chifflot, x='sizes', y='mkl', legend_label='mkl', color='blue', fill_alpha=0.4)
#p4.line(source=source_mkl_errors, x='sizes', y='mkl', line_color='blue')


p1.legend.location = "top_left"
p2.legend.location = "top_left"
p3.legend.location = "top_left"
p4.legend.location = "top_left"


show(gridplot([[p1, p2], [p3,p4]]))

## Enery Consumption

In [8]:
# --------------------------------------------------------------------------------------------------------------
# Data
# --------------------------------------------------------------------------------------------------------------

mkl_papi_chifflot = json.load(open('results/chifflot-8.lille.grid5000.fr_1/papi_mkl.json', 'r'))

cublas_cpu_papi_chifflot = json.load(open('results/chifflot-8.lille.grid5000.fr_1/papi_cublas_cpu.json', 'r'))
cublas_gpu_papi_chifflot = json.load(open('results/chifflot-8.lille.grid5000.fr_1/papi_cublas_gpu.json', 'r'))

cublasT_cpu_papi_chifflot = json.load(open('results/chifflot-8.lille.grid5000.fr_1/papi_cublasT_cpu.json', 'r'))
cublasT_gpu_papi_chifflot = json.load(open('results/chifflot-8.lille.grid5000.fr_1/papi_cublasT_gpu.json', 'r'))

sycl_cpu_papi_chifflot = json.load(open('results/chifflot-8.lille.grid5000.fr_1/papi_sycl_cpu.json', 'r'))
sycl_gpu_papi_chifflot = json.load(open('results/chifflot-8.lille.grid5000.fr_1/papi_sycl_gpu.json', 'r'))
sycl_cpu_gpu_papi_chifflot = json.load(open('results/chifflot-8.lille.grid5000.fr_1/papi_sycl_cpu_gpu.json', 'r'))

data_mkl_papi_chifflot = mkl_papi_chifflot['threads']['0']['regions']
data_cublas_cpu_papi_chifflot = cublas_cpu_papi_chifflot['threads']['0']['regions']
data_cublas_gpu_papi_chifflot = cublas_gpu_papi_chifflot['threads']['0']['regions']
data_cublasT_cpu_papi_chifflot = cublasT_cpu_papi_chifflot['threads']['0']['regions']
data_cublasT_gpu_papi_chifflot = cublasT_gpu_papi_chifflot['threads']['0']['regions']

data_sycl_cpu_papi_chifflot = sycl_cpu_papi_chifflot['threads']['0']['regions']

data_sycl_gpu_papi_chifflot = sycl_gpu_papi_chifflot['threads']['0']['regions']
data_sycl_cpu_gpu_papi_chifflot = sycl_cpu_gpu_papi_chifflot['threads']['0']['regions']

# --------------------------------------------------------------------------------------------------------------
# Notes:
# - Los datos se tomaron 50 veces, de debe promediar
# - Los datos de GPU esta en miliWatts
# - Los datos en CPU esta en nanoJoules
# --------------------------------------------------------------------------------------------------------------
iterations = 50

# --------------------------------------------------------------------------------------------------------------
# MKL
# --------------------------------------------------------------------------------------------------------------
sizes = [key for key, value in data_mkl_papi_chifflot.items()]
counters = [key for key, value in data_mkl_papi_chifflot['0'].items()]
i = 0
energy_0_mkl_papi_chifflot = list()
energy_1_mkl_papi_chifflot = list()
dram_energy_0_mkl_papi_chifflot = list()
dram_energy_1_mkl_papi_chifflot = list()
times_mkl_papi_chifflot = list()

for size in sizes:
    data_tmp = data_mkl_papi_chifflot[size]
    for counter in counters:
        if counter == 'rapl:::PACKAGE_ENERGY:PACKAGE0':
            energy_0_mkl_papi_chifflot.append(int(data_tmp[counter])*1e-9)
        if counter == 'rapl:::PACKAGE_ENERGY:PACKAGE1':
            energy_1_mkl_papi_chifflot.append(int(data_tmp[counter])*1e-9)
        if counter == 'rapl:::DRAM_ENERGY:PACKAGE0':
            dram_energy_0_mkl_papi_chifflot.append(int(data_tmp[counter])*1e-9)
        if counter == 'rapl:::DRAM_ENERGY:PACKAGE1':
            dram_energy_1_mkl_papi_chifflot.append(int(data_tmp[counter])*1e-9)
        if counter == 'real_time_nsec':
            times_mkl_papi_chifflot.append(int(data_tmp[counter])*1e-9)
    i = i + 1
# --------------------------------------------------------------------------------------------------------------
energy_0_mkl_papi_chifflot = [x / iterations for x in energy_0_mkl_papi_chifflot]
energy_1_mkl_papi_chifflot = [x / iterations for x in energy_1_mkl_papi_chifflot]
dram_energy_0_mkl_papi_chifflot = [x / iterations for x in dram_energy_0_mkl_papi_chifflot]
dram_energy_1_mkl_papi_chifflot = [x / iterations for x in dram_energy_1_mkl_papi_chifflot]

mkl_energy_cpu_papi_chifflot = [(x + y)/2 for (x, y) in zip(energy_0_mkl_papi_chifflot, energy_1_mkl_papi_chifflot)]
mkl_energy_dram_papi_chifflot = [(x + y)/2 for (x, y) in zip(dram_energy_0_mkl_papi_chifflot, dram_energy_1_mkl_papi_chifflot)]
mkl_energy_papi_chifflot = [x + y for (x, y) in zip(mkl_energy_cpu_papi_chifflot, mkl_energy_dram_papi_chifflot)] 

mkl_energy_w_papi_chifflot = [x / y for (x, y) in zip(mkl_energy_papi_chifflot, times_mkl_papi_chifflot)] 

# --------------------------------------------------------------------------------------------------------------
# cuBLAS
# --------------------------------------------------------------------------------------------------------------
# CPU
# --------------------------------------------------------------------------------------------------------------
sizes = [key for key, value in data_cublas_cpu_papi_chifflot.items()]
counters = [key for key, value in data_cublas_cpu_papi_chifflot['0'].items()]
i = 0
energy_0_cublas_cpu_papi_chifflot = list()
energy_1_cublas_cpu_papi_chifflot = list()
dram_energy_0_cublas_cpu_papi_chifflot = list()
dram_energy_1_cublas_cpu_papi_chifflot = list()
times_cublas_papi_chifflot = list()

for size in sizes:
    data_tmp = data_cublas_cpu_papi_chifflot[size]
    for counter in counters:     
        if counter == 'rapl:::PACKAGE_ENERGY:PACKAGE0':
            energy_0_cublas_cpu_papi_chifflot.append(int(data_tmp[counter])*1e-9)
        if counter == 'rapl:::PACKAGE_ENERGY:PACKAGE1':
            energy_1_cublas_cpu_papi_chifflot.append(int(data_tmp[counter])*1e-9)
        if counter == 'rapl:::DRAM_ENERGY:PACKAGE0':
            dram_energy_0_cublas_cpu_papi_chifflot.append(int(data_tmp[counter])*1e-9)
        if counter == 'rapl:::DRAM_ENERGY:PACKAGE1':
            dram_energy_1_cublas_cpu_papi_chifflot.append(int(data_tmp[counter])*1e-9)
        if counter == 'real_time_nsec':
            times_cublas_papi_chifflot.append(int(data_tmp[counter])*1e-9)
    i = i + 1
# --------------------------------------------------------------------------------------------------------------
# GPU
# --------------------------------------------------------------------------------------------------------------   
sizes = [key for key, value in data_cublas_gpu_papi_chifflot.items()]
counters = [key for key, value in data_cublas_gpu_papi_chifflot['0'].items()]
i = 0
energy_cublas_gpu_papi_chifflot = list()
times_cublas_gpu_papi_chifflot = list()

for size in sizes:
    data_tmp = data_cublas_gpu_papi_chifflot[size]
    for counter in counters:     
        if counter == 'nvml:::Tesla_V100-PCIE-32GB:device_0:power':
            energy_cublas_gpu_papi_chifflot.append(int(data_tmp[counter])*1e-3)
        if counter == 'real_time_nsec':
            times_cublas_gpu_papi_chifflot.append(int(data_tmp[counter])*1e-9)
    i = i + 1
# --------------------------------------------------------------------------------------------------------------
energy_0_cublas_cpu_papi_chifflot = [x / iterations for x in energy_0_cublas_cpu_papi_chifflot]
energy_1_cublas_cpu_papi_chifflot = [x / iterations for x in energy_1_cublas_cpu_papi_chifflot]
dram_energy_0_cublas_cpu_papi_chifflot = [x / iterations for x in dram_energy_0_cublas_cpu_papi_chifflot]
dram_energy_1_cublas_cpu_papi_chifflot = [x / iterations for x in dram_energy_1_cublas_cpu_papi_chifflot]

cublas_energy_cpu_papi_chifflot = [(x + y)/2 for (x, y) in zip(energy_0_cublas_cpu_papi_chifflot, energy_1_cublas_cpu_papi_chifflot)]
cublas_energy_dram_papi_chifflot = [(x + y)/2 for (x, y) in zip(dram_energy_0_cublas_cpu_papi_chifflot, dram_energy_1_cublas_cpu_papi_chifflot)]
cublas_energy_t_papi_chifflot = [x + y for (x, y) in zip(cublas_energy_cpu_papi_chifflot, cublas_energy_dram_papi_chifflot)]

energy_cublas_gpu_w_papi_chifflot = [x / iterations for x in energy_cublas_gpu_papi_chifflot]
energy_cublas_gpu_papi_chifflot = [x * y for (x, y) in zip(energy_cublas_gpu_w_papi_chifflot, times_cublas_gpu_papi_chifflot)]

cublas_energy_papi_chifflot = [x + y for (x, y) in zip(cublas_energy_t_papi_chifflot, energy_cublas_gpu_papi_chifflot)]


energy_0_cublas_w_papi_chifflot = [x / y for (x, y) in zip(energy_0_cublas_cpu_papi_chifflot, times_cublas_papi_chifflot)]
energy_1_cublas_w_papi_chifflot = [x / y for (x, y) in zip(energy_1_cublas_cpu_papi_chifflot, times_cublas_papi_chifflot)]
dram_energy_0_cublas_w_papi_chifflot = [x / y for (x, y) in zip(dram_energy_0_cublas_cpu_papi_chifflot, times_cublas_papi_chifflot)]
dram_energy_1_cublas_w_papi_chifflot = [x / y for (x, y) in zip(dram_energy_1_cublas_cpu_papi_chifflot, times_cublas_papi_chifflot)]

cublas_energy_w_papi_chifflot = [x / y for (x, y) in zip(cublas_energy_t_papi_chifflot, times_cublas_papi_chifflot)]
cublas_energy_w_papi_chifflot = [x + y for (x, y) in zip(cublas_energy_w_papi_chifflot, energy_cublas_gpu_w_papi_chifflot)] 

# --------------------------------------------------------------------------------------------------------------
# cuBLAS Tensor
# --------------------------------------------------------------------------------------------------------------
# CPU
# --------------------------------------------------------------------------------------------------------------
sizes = [key for key, value in data_cublasT_cpu_papi_chifflot.items()]
counters = [key for key, value in data_cublasT_cpu_papi_chifflot['0'].items()]
i = 0
energy_0_cublasT_cpu_papi_chifflot = list()
energy_1_cublasT_cpu_papi_chifflot = list()
dram_energy_0_cublasT_cpu_papi_chifflot = list()
dram_energy_1_cublasT_cpu_papi_chifflot = list()
times_cublasT_papi_chifflot = list()

for size in sizes:
    data_tmp = data_cublasT_cpu_papi_chifflot[size]
    for counter in counters:     
        if counter == 'rapl:::PACKAGE_ENERGY:PACKAGE0':
            energy_0_cublasT_cpu_papi_chifflot.append(int(data_tmp[counter])*1e-9)
        if counter == 'rapl:::PACKAGE_ENERGY:PACKAGE1':
            energy_1_cublasT_cpu_papi_chifflot.append(int(data_tmp[counter])*1e-9)
        if counter == 'rapl:::DRAM_ENERGY:PACKAGE0':
            dram_energy_0_cublasT_cpu_papi_chifflot.append(int(data_tmp[counter])*1e-9)
        if counter == 'rapl:::DRAM_ENERGY:PACKAGE1':
            dram_energy_1_cublasT_cpu_papi_chifflot.append(int(data_tmp[counter])*1e-9)
        if counter == 'real_time_nsec':
            times_cublasT_papi_chifflot.append(int(data_tmp[counter])*1e-9)
    i = i + 1

# --------------------------------------------------------------------------------------------------------------
# GPU
# --------------------------------------------------------------------------------------------------------------
sizes = [key for key, value in data_cublasT_gpu_papi_chifflot.items()]
counters = [key for key, value in data_cublasT_gpu_papi_chifflot['0'].items()]
i = 0
energy_cublasT_gpu_papi_chifflot = list()
times_cublasT_gpu_papi_chifflot = list()

for size in sizes:
    data_tmp = data_cublasT_gpu_papi_chifflot[size]
    for counter in counters:     
        if counter == 'nvml:::Tesla_V100-PCIE-32GB:device_0:power':
            energy_cublasT_gpu_papi_chifflot.append(int(data_tmp[counter])*1e-3)
        if counter == 'real_time_nsec':
            times_cublasT_gpu_papi_chifflot.append(int(data_tmp[counter])*1e-9)
    i = i + 1
# --------------------------------------------------------------------------------------------------------------
energy_0_cublasT_cpu_papi_chifflot = [x / iterations for x in energy_0_cublasT_cpu_papi_chifflot]
energy_1_cublasT_cpu_papi_chifflot = [x / iterations for x in energy_1_cublasT_cpu_papi_chifflot]
dram_energy_0_cublasT_cpu_papi_chifflot = [x / iterations for x in dram_energy_0_cublasT_cpu_papi_chifflot]
dram_energy_1_cublasT_cpu_papi_chifflot = [x / iterations for x in dram_energy_1_cublasT_cpu_papi_chifflot]

cublasT_energy_cpu_papi_chifflot = [(x + y)/2 for (x, y) in zip(energy_0_cublasT_cpu_papi_chifflot, energy_1_cublasT_cpu_papi_chifflot)]
cublasT_energy_dram_papi_chifflot = [(x + y)/2 for (x, y) in zip(dram_energy_0_cublasT_cpu_papi_chifflot, dram_energy_1_cublasT_cpu_papi_chifflot)]
cublasT_energy_t_papi_chifflot = [x + y for (x, y) in zip(cublasT_energy_cpu_papi_chifflot, cublasT_energy_dram_papi_chifflot)]

energy_cublasT_gpu_w_papi_chifflot = [x / iterations for x in energy_cublasT_gpu_papi_chifflot]
energy_cublasT_gpu_papi_chifflot = [x * y for (x, y) in zip(energy_cublasT_gpu_w_papi_chifflot, times_cublasT_gpu_papi_chifflot)]

cublasT_energy_papi_chifflot = [x + y for (x, y) in zip(cublasT_energy_t_papi_chifflot, energy_cublasT_gpu_papi_chifflot)]


energy_0_cublasT_w_papi_chifflot = [x / y for (x, y) in zip(energy_0_cublasT_cpu_papi_chifflot, times_cublasT_papi_chifflot)]
energy_1_cublasT_w_papi_chifflot = [x / y for (x, y) in zip(energy_1_cublasT_cpu_papi_chifflot, times_cublasT_papi_chifflot)]
dram_energy_0_cublasT_w_papi_chifflot = [x / y for (x, y) in zip(dram_energy_0_cublasT_cpu_papi_chifflot, times_cublasT_papi_chifflot)]
dram_energy_1_cublasT_w_papi_chifflot = [x / y for (x, y) in zip(dram_energy_1_cublasT_cpu_papi_chifflot, times_cublasT_papi_chifflot)]

cublasT_energy_w_papi_chifflot = [x / y for (x, y) in zip(cublasT_energy_t_papi_chifflot, times_cublasT_papi_chifflot)]
cublasT_energy_w_papi_chifflot = [x + y for (x, y) in zip(cublasT_energy_w_papi_chifflot, energy_cublasT_gpu_w_papi_chifflot)]    
# --------------------------------------------------------------------------------------------------------------
# SYCL CPU
# --------------------------------------------------------------------------------------------------------------
sizes = [key for key, value in data_sycl_cpu_papi_chifflot.items()]
counters = [key for key, value in data_sycl_cpu_papi_chifflot['0'].items()]
i = 0
energy_0_sycl_cpu_papi_chifflot = list()
energy_1_sycl_cpu_papi_chifflot = list()
dram_energy_0_sycl_cpu_papi_chifflot = list()
dram_energy_1_sycl_cpu_papi_chifflot = list()
times_sycl_cpu_papi_chifflot = list()

for size in sizes:
    data_tmp = data_sycl_cpu_papi_chifflot[size]
    for counter in counters:     
        if counter == 'rapl:::PACKAGE_ENERGY:PACKAGE0':
            energy_0_sycl_cpu_papi_chifflot.append(int(data_tmp[counter])*1e-9)
        if counter == 'rapl:::PACKAGE_ENERGY:PACKAGE1':
            energy_1_sycl_cpu_papi_chifflot.append(int(data_tmp[counter])*1e-9)
        if counter == 'rapl:::DRAM_ENERGY:PACKAGE0':
            dram_energy_0_sycl_cpu_papi_chifflot.append(int(data_tmp[counter])*1e-9)
        if counter == 'rapl:::DRAM_ENERGY:PACKAGE1':
            dram_energy_1_sycl_cpu_papi_chifflot.append(int(data_tmp[counter])*1e-9)
        if counter == 'real_time_nsec':
            times_sycl_cpu_papi_chifflot.append(int(data_tmp[counter])*1e-9)
    i = i + 1
# --------------------------------------------------------------------------------------------------------------
energy_0_sycl_cpu_papi_chifflot = [x / iterations for x in energy_0_sycl_cpu_papi_chifflot]
energy_1_sycl_cpu_papi_chifflot = [x / iterations for x in energy_1_sycl_cpu_papi_chifflot]
dram_energy_0_sycl_cpu_papi_chifflot = [x / iterations for x in dram_energy_0_sycl_cpu_papi_chifflot]
dram_energy_1_sycl_cpu_papi_chifflot = [x / iterations for x in dram_energy_1_sycl_cpu_papi_chifflot]

sycl_energy_cpu_papi_chifflot = [(x + y)/2 for (x, y) in zip(energy_0_sycl_cpu_papi_chifflot, energy_1_sycl_cpu_papi_chifflot)]
sycl_energy_dram_papi_chifflot = [(x + y)/2 for (x, y) in zip(dram_energy_0_sycl_cpu_papi_chifflot, dram_energy_1_sycl_cpu_papi_chifflot)]
sycl_energy_papi_chifflot = [x + y for (x, y) in zip(sycl_energy_cpu_papi_chifflot, sycl_energy_dram_papi_chifflot)] 

sycl_energy_w_papi_chifflot = [x / y for (x, y) in zip(sycl_energy_papi_chifflot, times_sycl_cpu_papi_chifflot)] 

# --------------------------------------------------------------------------------------------------------------
# SYCL 
# --------------------------------------------------------------------------------------------------------------
# GPU
# --------------------------------------------------------------------------------------------------------------
sizes = [key for key, value in data_sycl_gpu_papi_chifflot.items()]
counters = [key for key, value in data_sycl_gpu_papi_chifflot['0'].items()]
i = 0
energy_sycl_gpu_papi_chifflot = list()
times_sycl_gpu_papi_chifflot = list()

for size in sizes:
    data_tmp = data_sycl_gpu_papi_chifflot[size]
    for counter in counters:     
        if counter == 'nvml:::Tesla_V100-PCIE-32GB:device_0:power':
            energy_sycl_gpu_papi_chifflot.append(int(data_tmp[counter])*1e-3)
        if counter == 'real_time_nsec':
            times_sycl_gpu_papi_chifflot.append(int(data_tmp[counter])*1e-9)
    i = i + 1

# --------------------------------------------------------------------------------------------------------------
# CPU
# --------------------------------------------------------------------------------------------------------------
sizes = [key for key, value in data_sycl_cpu_gpu_papi_chifflot.items()]
counters = [key for key, value in data_sycl_cpu_gpu_papi_chifflot['0'].items()]
i = 0
energy_0_sycl_cpu_gpu_papi_chifflot = list()
energy_1_sycl_cpu_gpu_papi_chifflot = list()
dram_energy_0_sycl_cpu_gpu_papi_chifflot = list()
dram_energy_1_sycl_cpu_gpu_papi_chifflot = list()
times_sycl_cpu_gpu_papi_chifflot = list()

for size in sizes:
    data_tmp = data_sycl_cpu_gpu_papi_chifflot[size]
    for counter in counters:     
        if counter == 'rapl:::PACKAGE_ENERGY:PACKAGE0':
            energy_0_sycl_cpu_gpu_papi_chifflot.append(int(data_tmp[counter])*1e-9)
        if counter == 'rapl:::PACKAGE_ENERGY:PACKAGE1':
            energy_1_sycl_cpu_gpu_papi_chifflot.append(int(data_tmp[counter])*1e-9)
        if counter == 'rapl:::DRAM_ENERGY:PACKAGE0':
            dram_energy_0_sycl_cpu_gpu_papi_chifflot.append(int(data_tmp[counter])*1e-9)
        if counter == 'rapl:::DRAM_ENERGY:PACKAGE1':
            dram_energy_1_sycl_cpu_gpu_papi_chifflot.append(int(data_tmp[counter])*1e-9)
        if counter == 'real_time_nsec':
            times_sycl_cpu_gpu_papi_chifflot.append(int(data_tmp[counter])*1e-9)
    i = i + 1
# --------------------------------------------------------------------------------------------------------------
energy_0_sycl_cpu_gpu_papi_chifflot = [x / iterations for x in energy_0_sycl_cpu_gpu_papi_chifflot]
energy_1_sycl_cpu_gpu_papi_chifflot = [x / iterations for x in energy_1_sycl_cpu_gpu_papi_chifflot]
dram_energy_0_sycl_cpu_gpu_papi_chifflot = [x / iterations for x in dram_energy_0_sycl_cpu_gpu_papi_chifflot]
dram_energy_1_sycl_cpu_gpu_papi_chifflot = [x / iterations for x in dram_energy_1_sycl_cpu_gpu_papi_chifflot]

sycl_gpu_energy_cpu_papi_chifflot = [(x + y)/2 for (x, y) in zip(energy_0_sycl_cpu_gpu_papi_chifflot, energy_1_sycl_cpu_gpu_papi_chifflot)]
sycl_gpu_energy_dram_papi_chifflot = [(x + y)/2 for (x, y) in zip(dram_energy_0_sycl_cpu_gpu_papi_chifflot, dram_energy_1_sycl_cpu_gpu_papi_chifflot)]
sycl_gpu_energy_t_papi_chifflot = [x + y for (x, y) in zip(sycl_gpu_energy_cpu_papi_chifflot, sycl_gpu_energy_dram_papi_chifflot)]

energy_sycl_gpu_w_papi_chifflot = [x / iterations for x in energy_sycl_gpu_papi_chifflot]
energy_sycl_gpu_papi_chifflot = [x * y for (x, y) in zip(energy_sycl_gpu_w_papi_chifflot, times_sycl_gpu_papi_chifflot)]

sycl_gpu_energy_papi_chifflot = [x + y for (x, y) in zip(sycl_gpu_energy_t_papi_chifflot, energy_sycl_gpu_papi_chifflot)]


energy_0_sycl_w_papi_chifflot = [x / y for (x, y) in zip(energy_0_sycl_cpu_gpu_papi_chifflot, times_sycl_cpu_gpu_papi_chifflot)]
energy_1_sycl_w_papi_chifflot = [x / y for (x, y) in zip(energy_1_sycl_cpu_gpu_papi_chifflot, times_sycl_cpu_gpu_papi_chifflot)]
dram_energy_0_sycl_w_papi_chifflot = [x / y for (x, y) in zip(dram_energy_0_sycl_cpu_gpu_papi_chifflot, times_sycl_cpu_gpu_papi_chifflot)]
dram_energy_1_sycl_w_papi_chifflot = [x / y for (x, y) in zip(dram_energy_1_sycl_cpu_gpu_papi_chifflot, times_sycl_cpu_gpu_papi_chifflot)]

sycl_gpu_energy_w_papi_chifflot = [x / y for (x, y) in zip(sycl_gpu_energy_t_papi_chifflot, times_sycl_cpu_gpu_papi_chifflot)]
sycl_gpu_energy_w_papi_chifflot = [x + y for (x, y) in zip(sycl_gpu_energy_w_papi_chifflot, energy_sycl_gpu_w_papi_chifflot)]    

In [9]:
# --------------------------------------------------------------------------------------------------------------
# Data source
# --------------------------------------------------------------------------------------------------------------
list_size = list([32, 48, 64, 80, 96, 112, 128, 256, 384, 512, 640, 768, 896, 1024, 2048, 3072, 4096, 5120, 6144, 7168, 8192])

# --------------------------------------------------------------------------------------------------------------
# MKL
# --------------------------------------------------------------------------------------------------------------
data_energy_mkl_papi_chifflot = pd.DataFrame({'size':list_size,'PACKAGE_ENERGY0':energy_0_mkl_papi_chifflot, 'PACKAGE_ENERGY1':energy_1_mkl_papi_chifflot, 'DRAM_ENERGY0':dram_energy_0_mkl_papi_chifflot, 'DRAM_ENERGY1':dram_energy_1_mkl_papi_chifflot})
source_energy_mkl_papi_chifflot = ColumnDataSource(data_energy_mkl_papi_chifflot)
# --------------------------------------------------------------------------------------------------------------
data_energy_mkl_total_papi_chifflot = pd.DataFrame({'size':list_size, 'POWER':mkl_energy_papi_chifflot})
source_energy_mkl_total_papi_chifflot = ColumnDataSource(data_energy_mkl_total_papi_chifflot)

# --------------------------------------------------------------------------------------------------------------
# cuBLAS
# --------------------------------------------------------------------------------------------------------------
# CPU
# --------------------------------------------------------------------------------------------------------------
data_energy_cublas_cpu_papi_chifflot = pd.DataFrame({'size':list_size,'PACKAGE_ENERGY0':energy_0_cublas_cpu_papi_chifflot, 'PACKAGE_ENERGY1':energy_1_cublas_cpu_papi_chifflot, 'DRAM_ENERGY0':dram_energy_0_cublas_cpu_papi_chifflot, 'DRAM_ENERGY1':dram_energy_1_cublas_cpu_papi_chifflot})
source_energy_cublas_cpu_papi_chifflot = ColumnDataSource(data_energy_cublas_cpu_papi_chifflot)
# --------------------------------------------------------------------------------------------------------------
# GPU
# --------------------------------------------------------------------------------------------------------------   
data_energy_cublas_gpu_papi_chifflot = pd.DataFrame({'size':list_size,'POWER':energy_cublas_gpu_papi_chifflot})
source_energy_cublas_gpu_papi_chifflot = ColumnDataSource(data_energy_cublas_gpu_papi_chifflot)
# --------------------------------------------------------------------------------------------------------------   
data_energy_cublas_total_papi_chifflot = pd.DataFrame({'size':list_size, 'POWER':cublas_energy_papi_chifflot})
source_energy_cublas_total_papi_chifflot = ColumnDataSource(data_energy_cublas_total_papi_chifflot)


# --------------------------------------------------------------------------------------------------------------
# cuBLAS Tensor
# --------------------------------------------------------------------------------------------------------------
# CPU
# --------------------------------------------------------------------------------------------------------------
data_energy_cublasT_cpu_papi_chifflot = pd.DataFrame({'size':list_size,'PACKAGE_ENERGY0':energy_0_cublasT_cpu_papi_chifflot, 'PACKAGE_ENERGY1':energy_1_cublasT_cpu_papi_chifflot, 'DRAM_ENERGY0':dram_energy_0_cublasT_cpu_papi_chifflot, 'DRAM_ENERGY1':dram_energy_1_cublasT_cpu_papi_chifflot})
source_energy_cublasT_cpu_papi_chifflot = ColumnDataSource(data_energy_cublasT_cpu_papi_chifflot)
# --------------------------------------------------------------------------------------------------------------
# GPU
# --------------------------------------------------------------------------------------------------------------
data_energy_cublasT_gpu_papi_chifflot = pd.DataFrame({'size':list_size,'POWER':energy_cublasT_gpu_papi_chifflot})
source_energy_cublasT_gpu_papi_chifflot = ColumnDataSource(data_energy_cublasT_gpu_papi_chifflot)
# --------------------------------------------------------------------------------------------------------------   
data_energy_cublasT_total_papi_chifflot = pd.DataFrame({'size':list_size, 'POWER':cublasT_energy_papi_chifflot})
source_energy_cublasT_total_papi_chifflot = ColumnDataSource(data_energy_cublasT_total_papi_chifflot)

# --------------------------------------------------------------------------------------------------------------
# SYCL CPU
# --------------------------------------------------------------------------------------------------------------
data_energy_sycl_cpu_papi_chifflot = pd.DataFrame({'size':list_size,'PACKAGE_ENERGY0':energy_0_sycl_cpu_papi_chifflot, 'PACKAGE_ENERGY1':energy_1_sycl_cpu_papi_chifflot, 'DRAM_ENERGY0':dram_energy_0_sycl_cpu_papi_chifflot, 'DRAM_ENERGY1':dram_energy_1_sycl_cpu_papi_chifflot})
source_energy_sycl_cpu_papi_chifflot = ColumnDataSource(data_energy_sycl_cpu_papi_chifflot)
# --------------------------------------------------------------------------------------------------------------
data_energy_sycl_total_papi_chifflot = pd.DataFrame({'size':list_size, 'POWER':sycl_energy_papi_chifflot})
source_energy_sycl_total_papi_chifflot = ColumnDataSource(data_energy_sycl_total_papi_chifflot)

# --------------------------------------------------------------------------------------------------------------
# SYCL 
# --------------------------------------------------------------------------------------------------------------
# GPU
# --------------------------------------------------------------------------------------------------------------
data_energy_sycl_gpu_papi_chifflot = pd.DataFrame({'size':list_size,'POWER':energy_sycl_gpu_papi_chifflot})
source_energy_sycl_gpu_papi_chifflot = ColumnDataSource(data_energy_sycl_gpu_papi_chifflot)
# --------------------------------------------------------------------------------------------------------------
# CPU
# --------------------------------------------------------------------------------------------------------------
data_energy_sycl_cpu_gpu_papi_chifflot = pd.DataFrame({'size':list_size,'PACKAGE_ENERGY0':energy_0_sycl_cpu_gpu_papi_chifflot, 'PACKAGE_ENERGY1':energy_1_sycl_cpu_gpu_papi_chifflot, 'DRAM_ENERGY0':dram_energy_0_sycl_cpu_gpu_papi_chifflot, 'DRAM_ENERGY1':dram_energy_1_sycl_cpu_gpu_papi_chifflot})
source_energy_sycl_cpu_gpu_papi_chifflot = ColumnDataSource(data_energy_sycl_cpu_gpu_papi_chifflot)
# --------------------------------------------------------------------------------------------------------------   
data_energy_sycl_gpu_total_papi_chifflot = pd.DataFrame({'size':list_size, 'POWER':sycl_gpu_energy_papi_chifflot})
source_energy_sycl_gpu_total_papi_chifflot = ColumnDataSource(data_energy_sycl_gpu_total_papi_chifflot)


In [10]:
# --------------------------------------------------------------------------------------------------------------
# Graph
# --------------------------------------------------------------------------------------------------------------
p1 = figure(title="Energy MKL", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p1.xaxis.axis_label = 'Matrix Size (N x N)'
p1.yaxis.axis_label = 'Energy ( Joules )'

p2 = figure(title="Energy MKL (Average)", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p2.xaxis.axis_label = 'Matrix Size (N x N)'
p2.yaxis.axis_label = 'Energy ( Joules )'

p3 = figure(title="Energy cuBLAS", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p3.xaxis.axis_label = 'Matrix Size (N x N)'
p3.yaxis.axis_label = 'Energy ( Joules )'

p4 = figure(title="Energy cuBLAS (Average)", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p4.xaxis.axis_label = 'Matrix Size (N x N)'
p4.yaxis.axis_label = 'Energy ( Joules )'

p5 = figure(title="Energy cuBLAS Tensor", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p5.xaxis.axis_label = 'Matrix Size (N x N)'
p5.yaxis.axis_label = 'Energy ( Joules )'

p6 = figure(title="Energy cuBLAS Tensor (Average)", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p6.xaxis.axis_label = 'Matrix Size (N x N)'
p6.yaxis.axis_label = 'Energy ( Joules )'

p7 = figure(title="Energy SYCL", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p7.xaxis.axis_label = 'Matrix Size (N x N)'
p7.yaxis.axis_label = 'Energy ( Joules )'

p8 = figure(title="Energy SYCL (Average)", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p8.xaxis.axis_label = 'Matrix Size (N x N)'
p8.yaxis.axis_label = 'Energy ( Joules )'

p9 = figure(title="Energy SYCL/GPU", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p9.xaxis.axis_label = 'Matrix Size (N x N)'
p9.yaxis.axis_label = 'Energy ( Joules )'

p10 = figure(title="Energy SYCL/GPU (Average)", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p10.xaxis.axis_label = 'Matrix Size (N x N)'
p10.yaxis.axis_label = 'Energy ( Joules )'

# --------------------------------------------------------------------------------------------------------------
# MKL
# --------------------------------------------------------------------------------------------------------------
p1.square(source=source_energy_mkl_papi_chifflot, x='size', y='PACKAGE_ENERGY0', legend_label='PACKAGE_ENERGY0_MKL', fill_color=None, line_color='orangered')
p1.line(source=source_energy_mkl_papi_chifflot, x='size', y='PACKAGE_ENERGY0', line_color='orangered')
p1.square(source=source_energy_mkl_papi_chifflot, x='size', y='PACKAGE_ENERGY1', legend_label='PACKAGE_ENERGY1_MKL', fill_color=None, line_color='orange')
p1.line(source=source_energy_mkl_papi_chifflot, x='size', y='PACKAGE_ENERGY1', line_color='orange')
p1.square(source=source_energy_mkl_papi_chifflot, x='size', y='DRAM_ENERGY0', legend_label='DRAM_ENERGY0_MKL', fill_color=None, line_color='mediumseagreen')
p1.line(source=source_energy_mkl_papi_chifflot, x='size', y='DRAM_ENERGY0', line_color='mediumseagreen')
p1.square(source=source_energy_mkl_papi_chifflot, x='size', y='DRAM_ENERGY1', legend_label='DRAM_ENERGY1_MKL', fill_color=None, line_color='teal')
p1.line(source=source_energy_mkl_papi_chifflot, x='size', y='DRAM_ENERGY1', line_color='teal')

p2.square(source=source_energy_mkl_total_papi_chifflot, x='size', y='POWER', legend_label='MKL - TOTAL POWER', fill_color=None, line_color='midnightblue')
p2.line(source=source_energy_mkl_total_papi_chifflot, x='size', y='POWER', line_color='midnightblue')

# --------------------------------------------------------------------------------------------------------------
# cuBLAS
# --------------------------------------------------------------------------------------------------------------
p3.square(source=source_energy_cublas_cpu_papi_chifflot, x='size', y='PACKAGE_ENERGY0', legend_label='PACKAGE_ENERGY0_cuBLAS_cpu', fill_color=None, line_color='orangered')
p3.line(source=source_energy_cublas_cpu_papi_chifflot, x='size', y='PACKAGE_ENERGY0', line_color='orangered')
p3.square(source=source_energy_cublas_cpu_papi_chifflot, x='size', y='PACKAGE_ENERGY1', legend_label='PACKAGE_ENERGY1_cuBLAS_cpu', fill_color=None, line_color='orange')
p3.line(source=source_energy_cublas_cpu_papi_chifflot, x='size', y='PACKAGE_ENERGY1', line_color='orange')
p3.square(source=source_energy_cublas_cpu_papi_chifflot, x='size', y='DRAM_ENERGY0', legend_label='DRAM_ENERGY0_cuBLAS_cpu', fill_color=None, line_color='mediumseagreen')
p3.line(source=source_energy_cublas_cpu_papi_chifflot, x='size', y='DRAM_ENERGY0', line_color='mediumseagreen')
p3.square(source=source_energy_cublas_cpu_papi_chifflot, x='size', y='DRAM_ENERGY1', legend_label='DRAM_ENERGY1_cuBLAS_cpu', fill_color=None, line_color='teal')
p3.line(source=source_energy_cublas_cpu_papi_chifflot, x='size', y='DRAM_ENERGY1', line_color='teal')
p3.square(source=source_energy_cublas_gpu_papi_chifflot, x='size', y='POWER', legend_label='GPU cuBLAS', fill_color=None, line_color='brown')
p3.line(source=source_energy_cublas_gpu_papi_chifflot, x='size', y='POWER', line_color='brown')

p4.square(source=source_energy_cublas_total_papi_chifflot, x='size', y='POWER', legend_label='cuBLAS - TOTAL POWER', fill_color=None, line_color='mediumseagreen')
p4.line(source=source_energy_cublas_total_papi_chifflot, x='size', y='POWER', line_color='mediumseagreen')

# --------------------------------------------------------------------------------------------------------------
# cuBLAS Tensor
# --------------------------------------------------------------------------------------------------------------
p5.square(source=source_energy_cublasT_cpu_papi_chifflot, x='size', y='PACKAGE_ENERGY0', legend_label='PACKAGE_ENERGY0_cuBLAS_Tensor_cpu', fill_color=None, line_color='orangered')
p5.line(source=source_energy_cublasT_cpu_papi_chifflot, x='size', y='PACKAGE_ENERGY0', line_color='orangered')
p5.square(source=source_energy_cublasT_cpu_papi_chifflot, x='size', y='PACKAGE_ENERGY1', legend_label='PACKAGE_ENERGY1_cuBLAS_Tensor_cpu', fill_color=None, line_color='orange')
p5.line(source=source_energy_cublasT_cpu_papi_chifflot, x='size', y='PACKAGE_ENERGY1', line_color='orange')
p5.square(source=source_energy_cublasT_cpu_papi_chifflot, x='size', y='DRAM_ENERGY0', legend_label='DRAM_ENERGY0_cuBLAS_Tensor_cpu', fill_color=None, line_color='mediumseagreen')
p5.line(source=source_energy_cublasT_cpu_papi_chifflot, x='size', y='DRAM_ENERGY0', line_color='mediumseagreen')
p5.square(source=source_energy_cublasT_cpu_papi_chifflot, x='size', y='DRAM_ENERGY1', legend_label='DRAM_ENERGY1_cuBLAS_Tensor_cpu', fill_color=None, line_color='teal')
p5.line(source=source_energy_cublasT_cpu_papi_chifflot, x='size', y='DRAM_ENERGY1', line_color='teal')
p5.square(source=source_energy_cublasT_gpu_papi_chifflot, x='size', y='POWER', legend_label='GPU cuBLAS Tensor', fill_color=None, line_color='brown')
p5.line(source=source_energy_cublasT_gpu_papi_chifflot, x='size', y='POWER', line_color='brown')

p6.square(source=source_energy_cublasT_total_papi_chifflot, x='size', y='POWER', legend_label='cuBLAS Tensor - TOTAL POWER', fill_color=None, line_color='tomato')
p6.line(source=source_energy_cublasT_total_papi_chifflot, x='size', y='POWER', line_color='tomato')

# --------------------------------------------------------------------------------------------------------------
# SYCL CPU
# --------------------------------------------------------------------------------------------------------------
p7.square(source=source_energy_sycl_cpu_papi_chifflot, x='size', y='PACKAGE_ENERGY0', legend_label='PACKAGE_ENERGY0_SYCL', fill_color=None, line_color='orangered')
p7.line(source=source_energy_sycl_cpu_papi_chifflot, x='size', y='PACKAGE_ENERGY0', line_color='orangered')
p7.square(source=source_energy_sycl_cpu_papi_chifflot, x='size', y='PACKAGE_ENERGY1', legend_label='PACKAGE_ENERGY1_SYCL', fill_color=None, line_color='orange')
p7.line(source=source_energy_sycl_cpu_papi_chifflot, x='size', y='PACKAGE_ENERGY1', line_color='orange')
p7.square(source=source_energy_sycl_cpu_papi_chifflot, x='size', y='DRAM_ENERGY0', legend_label='DRAM_ENERGY0_SYCL', fill_color=None, line_color='mediumseagreen')
p7.line(source=source_energy_sycl_cpu_papi_chifflot, x='size', y='DRAM_ENERGY0', line_color='mediumseagreen')
p7.square(source=source_energy_sycl_cpu_papi_chifflot, x='size', y='DRAM_ENERGY1', legend_label='DRAM_ENERGY1_SYCL', fill_color=None, line_color='teal')
p7.line(source=source_energy_sycl_cpu_papi_chifflot, x='size', y='DRAM_ENERGY1', line_color='teal')

p8.square(source=source_energy_sycl_total_papi_chifflot, x='size', y='POWER', legend_label='SYCL - TOTAL POWER', fill_color=None, line_color='darkturquoise')
p8.line(source=source_energy_sycl_total_papi_chifflot, x='size', y='POWER', line_color='darkturquoise')

# --------------------------------------------------------------------------------------------------------------
# SYCL GPU
# --------------------------------------------------------------------------------------------------------------
p9.square(source=source_energy_sycl_cpu_gpu_papi_chifflot, x='size', y='PACKAGE_ENERGY0', legend_label='PACKAGE_ENERGY0_SYCL_GPU', fill_color=None, line_color='orangered')
p9.line(source=source_energy_sycl_cpu_gpu_papi_chifflot, x='size', y='PACKAGE_ENERGY0', line_color='orangered')
p9.square(source=source_energy_sycl_cpu_gpu_papi_chifflot, x='size', y='PACKAGE_ENERGY1', legend_label='PACKAGE_ENERGY1_SYCL_GPU', fill_color=None, line_color='orange')
p9.line(source=source_energy_sycl_cpu_gpu_papi_chifflot, x='size', y='PACKAGE_ENERGY1', line_color='orange')
p9.square(source=source_energy_sycl_cpu_gpu_papi_chifflot, x='size', y='DRAM_ENERGY0', legend_label='DRAM_ENERGY0_SYCL_GPU', fill_color=None, line_color='mediumseagreen')
p9.line(source=source_energy_sycl_cpu_gpu_papi_chifflot, x='size', y='DRAM_ENERGY0', line_color='mediumseagreen')
p9.square(source=source_energy_sycl_cpu_gpu_papi_chifflot, x='size', y='DRAM_ENERGY1', legend_label='DRAM_ENERGY1_SYCL_GPU', fill_color=None, line_color='teal')
p9.line(source=source_energy_sycl_cpu_gpu_papi_chifflot, x='size', y='DRAM_ENERGY1', line_color='teal')
p9.square(source=source_energy_sycl_gpu_papi_chifflot, x='size', y='POWER', legend_label='GPU SYCL_GPU', fill_color=None, line_color='brown')
p9.line(source=source_energy_sycl_gpu_papi_chifflot, x='size', y='POWER', line_color='brown')

p10.square(source=source_energy_sycl_gpu_total_papi_chifflot, x='size', y='POWER', legend_label='SYCL_GPU - TOTAL POWER', fill_color=None, line_color='mediumpurple')
p10.line(source=source_energy_sycl_gpu_total_papi_chifflot, x='size', y='POWER', line_color='mediumpurple')
# --------------------------------------------------------------------------------------------------------------


p1.legend.location = "top_left"
p2.legend.location = "top_left"
p3.legend.location = "top_left"
p4.legend.location = "top_left"
p5.legend.location = "top_left"
p6.legend.location = "top_left"
p7.legend.location = "top_left"
p8.legend.location = "top_left"
p9.legend.location = "top_left"
p10.legend.location = "top_left"

show(gridplot([[p1, p2], [p3,p4], [p5,p6], [p7, p8], [p9, p10]]))

In [11]:
p1 = figure(title="Energy Average", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p2 = figure(title="Energy Average", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'Matrix Size (N x N)'
p1.yaxis.axis_label = 'Energy ( Joules )'
p2.xaxis.axis_label = 'Matrix Size (N x N)'
p2.yaxis.axis_label = 'Energy ( Joules )'

p1.square(source=source_energy_mkl_total_papi_chifflot, x='size', y='POWER', legend_label='MKL - TOTAL POWER', fill_color=None, line_color='midnightblue')
p1.line(source=source_energy_mkl_total_papi_chifflot, x='size', y='POWER', line_color='midnightblue')
p1.square(source=source_energy_cublas_total_papi_chifflot, x='size', y='POWER', legend_label='cuBLAS - TOTAL POWER', fill_color=None, line_color='mediumseagreen')
p1.line(source=source_energy_cublas_total_papi_chifflot, x='size', y='POWER', line_color='mediumseagreen')
p1.square(source=source_energy_cublasT_total_papi_chifflot, x='size', y='POWER', legend_label='cuBLAS Tensor - TOTAL POWER', fill_color=None, line_color='tomato')
p1.line(source=source_energy_cublasT_total_papi_chifflot, x='size', y='POWER', line_color='tomato')
p1.square(source=source_energy_sycl_total_papi_chifflot, x='size', y='POWER', legend_label='SYCL - TOTAL POWER', fill_color=None, line_color='darkturquoise')
p1.line(source=source_energy_sycl_total_papi_chifflot, x='size', y='POWER', line_color='darkturquoise')
p1.square(source=source_energy_sycl_gpu_total_papi_chifflot, x='size', y='POWER', legend_label='SYCL_GPU - TOTAL POWER', fill_color=None, line_color='mediumpurple')
p1.line(source=source_energy_sycl_gpu_total_papi_chifflot, x='size', y='POWER', line_color='mediumpurple')

p2.square(source=source_energy_mkl_total_papi_chifflot, x='size', y='POWER', legend_label='MKL - TOTAL POWER', fill_color=None, line_color='midnightblue')
p2.line(source=source_energy_mkl_total_papi_chifflot, x='size', y='POWER', line_color='midnightblue')
p2.square(source=source_energy_cublasT_total_papi_chifflot, x='size', y='POWER', legend_label='cuBLAS Tensor - TOTAL POWER', fill_color=None, line_color='tomato')
p2.line(source=source_energy_cublasT_total_papi_chifflot, x='size', y='POWER', line_color='tomato')

p1.legend.location = "top_left"
p2.legend.location = "top_left"

show(gridplot([p1, p2], ncols=2))

## Power consumption

In [12]:
p1 = figure(title="Power Consumption", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p2 = figure(title="Power Consumption", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'Matrix Size (N x N)'
p1.yaxis.axis_label = 'Power Consumption ( Watts )'
p2.xaxis.axis_label = 'Matrix Size (N x N)'
p2.yaxis.axis_label = 'Power Consumption ( Watts )'


data_energy_mkl_w_papi_chifflot = pd.DataFrame({'size':list_size,'ENERGY':mkl_energy_w_papi_chifflot})
source_mkl_w_papi_chifflot = ColumnDataSource(data_energy_mkl_w_papi_chifflot)

data_energy_cublas_w_papi_chifflot = pd.DataFrame({'size':list_size,'ENERGY':cublas_energy_w_papi_chifflot})
source_cublas_w_papi_chifflot = ColumnDataSource(data_energy_cublas_w_papi_chifflot)

data_energy_cublasT_w_papi_chifflot = pd.DataFrame({'size':list_size,'ENERGY':cublasT_energy_w_papi_chifflot})
source_cublasT_w_papi_chifflot = ColumnDataSource(data_energy_cublasT_w_papi_chifflot)

data_energy_sycl_w_papi_chifflot = pd.DataFrame({'size':list_size,'ENERGY':sycl_energy_w_papi_chifflot})
source_sycl_w_papi_chifflot = ColumnDataSource(data_energy_sycl_w_papi_chifflot)

data_energy_sycl_gpu_w_papi_chifflot = pd.DataFrame({'size':list_size,'ENERGY':sycl_gpu_energy_w_papi_chifflot})
source_sycl_gpu_w_papi_chifflot = ColumnDataSource(data_energy_sycl_gpu_w_papi_chifflot)

p1.square(source=source_mkl_w_papi_chifflot, x='size', y='ENERGY', legend_label='mkl', color='midnightblue', fill_alpha=0.4)
p1.square(source=source_cublas_w_papi_chifflot, x='size', y='ENERGY', legend_label='cuBLAS', color='mediumseagreen', fill_alpha=0.4)
p1.square(source=source_cublasT_w_papi_chifflot, x='size', y='ENERGY', legend_label='cuBLAS Tensor', color='tomato', fill_alpha=0.4)
p1.square(source=source_sycl_w_papi_chifflot, x='size', y='ENERGY', legend_label='SYCL', color='darkturquoise', fill_alpha=0.4)
p1.square(source=source_sycl_gpu_w_papi_chifflot, x='size', y='ENERGY', legend_label='SYCL GPU', color='mediumpurple', fill_alpha=0.4)

p2.square(source=source_mkl_w_papi_chifflot, x='size', y='ENERGY', legend_label='mkl', color='midnightblue', fill_alpha=0.4)
p2.square(source=source_cublasT_w_papi_chifflot, x='size', y='ENERGY', legend_label='cuBLAS Tensor', color='tomato', fill_alpha=0.4)


p1.legend.location = "bottom_right"
p2.legend.location = "bottom_right"


show(gridplot([p1,p2], ncols=2))

## Final

In [13]:
p1 = figure(title="Execution Times", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p2 = figure(title="Error", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p3 = figure(title="Energy Consumption", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p4 = figure(title="Power Consumption", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")


p1.xaxis.axis_label = 'Matrix Size (N x N)'
p1.yaxis.axis_label = 'Time ( ms )'
p2.xaxis.axis_label = 'Matrix Size (N x N)'
p2.yaxis.axis_label = 'MSE'
p3.xaxis.axis_label = 'Matrix Size (N x N)'
p3.yaxis.axis_label = 'Energy ( Joules )'
p4.xaxis.axis_label = 'Matrix Size (N x N)'
p4.yaxis.axis_label = 'Power Consumption ( Watts )'

p1.square(source=source_cublas_chifflot, x='size', y='time', legend_label='cuBLAS', fill_color=None, line_color='mediumseagreen')
p1.line(source=source_cublas_chifflot, x='size', y='time', line_color='mediumseagreen')
p1.square(source=source_cublasT_chifflot, x='size', y='time', legend_label='cuBLAS_Tensor', fill_color=None, line_color='tomato')
p1.line(source=source_cublasT_chifflot, x='size', y='time', line_color='tomato')
p1.square(source=source_mkl_chifflot, x='size', y='time', legend_label='mkl', fill_color=None, line_color='blue')
p1.line(source=source_mkl_chifflot, x='size', y='time', line_color='blue')

p2.square(source=source_cublas_errors_chifflot, x='sizes', y='cuBLAS', legend_label='cuBLAS', color='mediumseagreen', fill_alpha=0.4)
p2.square(source=source_cublasT_errors_chifflot, x='sizes', y='cuBLAS_Tensor', legend_label='cuBLAS_Tensor', color='tomato', fill_alpha=0.4)
#p2.line(source=source_cublasT_errors, x='sizes', y='cuBLAS_Tensor', line_color='tomato')
p2.square(source=source_mkl_errors_chifflot, x='sizes', y='mkl', legend_label='mkl', color='blue', fill_alpha=0.4)
#p2.line(source=source_mkl_errors, x='sizes', y='mkl', line_color='blue')

p3.square(source=source_energy_mkl_total_papi_chifflot, x='size', y='POWER', legend_label='MKL - TOTAL POWER', fill_color=None, line_color='midnightblue')
p3.line(source=source_energy_mkl_total_papi_chifflot, x='size', y='POWER', line_color='midnightblue')
p3.square(source=source_energy_cublasT_total_papi_chifflot, x='size', y='POWER', legend_label='cuBLAS Tensor - TOTAL POWER', fill_color=None, line_color='tomato')
p3.line(source=source_energy_cublasT_total_papi_chifflot, x='size', y='POWER', line_color='tomato')
p3.square(source=source_energy_cublas_total_papi_chifflot, x='size', y='POWER', legend_label='cuBLAS - TOTAL POWER', fill_color=None, line_color='mediumseagreen')
p3.line(source=source_energy_cublas_total_papi_chifflot, x='size', y='POWER', line_color='mediumseagreen')


p4.square(source=source_mkl_w_papi_chifflot, x='size', y='ENERGY', legend_label='mkl', color='midnightblue', fill_alpha=0.4)
p4.square(source=source_cublas_w_papi_chifflot, x='size', y='ENERGY', legend_label='cuBLAS', color='mediumseagreen', fill_alpha=0.4)
p4.square(source=source_cublasT_w_papi_chifflot, x='size', y='ENERGY', legend_label='cuBLAS Tensor', color='tomato', fill_alpha=0.4)


p1.legend.location = "top_left"
p2.legend.location = "top_left"
p3.legend.location = "top_left"
p4.legend.location = "top_left"

show(gridplot([p1, p2, p3, p4], ncols=4))

___
## roazhon13-1.rennes.grid5000.fr
### Performace -CPU

In [14]:
data = json.load(open('results/roazhon13-1.rennes.grid5000.fr/results_cpu.json', 'r'))
labels =  [key for key, value in data.items()]
i = 0

colors = ['green', 'red', 'blue', 'darkturquoise', 'coral']

p1 = figure(title="Execution Times - CPU", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p2 = figure(title="Error - CPU", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'Matrix size (N x N)'
p1.yaxis.axis_label = 'Time ( ms )'
p2.xaxis.axis_label = 'Matrix size (N x N)'
p2.yaxis.axis_label = 'MSE'

errors_total = list()

for label in labels:
    times = list()
    errors = list()
    data_err = data[label]
    sizes = [key for key, value in data_err.items()]
    for size in sizes:
        times.append(data_err[size]['time'])
        errors.append(data_err[size]['error'])
    sizes = list(map(int, sizes))

    data_plot = pd.DataFrame({'size':sizes, 'time':times, 'error':errors})
    # Add error bars to the DataFrame
    errors_total += [errors]
    errors = list(map(float, errors))
    errors_t = dict()
    errors_t['sizes'] = sizes
    for key, value in zip(labels, errors_total):
        errors_t[key] = value
    source_cpu_errors = ColumnDataSource(pd.DataFrame(errors_t))

    
    data_plot['error_low'] = data_plot['time'] - np.array(errors)
    data_plot['error_high'] = data_plot['time'] + np.array(errors)
    data_plot = data_plot.sort_values(by=['size'], ascending = True)
    data_plot = data_plot.reset_index(drop=True)
    source_cpu = ColumnDataSource(data_plot)
    
    p1.square(source=source_cpu, x='size', y='time', legend_label=labels[i], fill_color=None, line_color=colors[i])
    p1.line(source=source_cpu, x='size', y='time', line_color=colors[i])
    
    if label != 'serial':
        #p2.line(source=source_cpu_errors, x='sizes', y=label, legend_label=labels[i], color=colors[i])
        p2.square(source=source_cpu_errors, x='sizes', y=label, legend_label=labels[i], color=colors[i])
    
    if label == 'mkl':
        data_mkl_roazhon13 = data_plot.copy(deep = True)
        error_mkl_roazhon13 = pd.DataFrame(errors_t)
        
    if label == 'openMP':
        data_openMP_roazhon13 = data_plot.copy(deep = True)
        error_openMP_roazhon13 = pd.DataFrame(errors_t)
        
    i=i+1

maxi = max(max([sublist[:] for sublist in errors_total]))


p2.y_range.end = maxi*1.1

p1.legend.location = "top_left"
p2.legend.location = "top_left"

show(gridplot([p1, p2], ncols=2))

In [15]:
p1 = figure(title="Execution Times - CPU", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p2 = figure(title="Error - CPU", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'Matrix size (N x N)'
p1.yaxis.axis_label = 'Time ( ms )'

p2.xaxis.axis_label = 'Matrix size (N x N)'
p2.yaxis.axis_label = 'MSE'

source_mkl_roazhon13 = ColumnDataSource(data_mkl_roazhon13)
source_mkl_errors_roazhon13 = ColumnDataSource(error_mkl_roazhon13)
source_openmp_roazhon13 = ColumnDataSource(data_openMP_roazhon13)
source_openmp_errors_roazhon13 = ColumnDataSource(error_openMP_roazhon13)

p1.square(source=source_mkl_roazhon13, x='size', y='time', legend_label='mkl', fill_color=None, line_color='blue')
p1.line(source=source_mkl_roazhon13, x='size', y='time', line_color='blue')
p1.square(source=source_openmp_roazhon13, x='size', y='time', legend_label='openMP', fill_color=None, line_color='darkturquoise')
p1.line(source=source_openmp_roazhon13, x='size', y='time', line_color='darkturquoise')

p1.legend.location = "top_left"

p2.square(source=source_mkl_errors_roazhon13, x='sizes', y='mkl', legend_label='mkl', color='blue', fill_alpha=0.4)
#p2.line(source=source_mkl_errors, x='sizes', y='mkl', line_color='blue')
p2.square(source=source_openmp_errors_roazhon13, x='sizes', y='openMP', legend_label='openMP', color='darkturquoise', fill_alpha=0.4)
#p2.line(source=source_openmp_errors, x='sizes', y='openMP', line_color='darkturquoise')
p2.y_range.end = maxi*1.1
p2.legend.location = "top_left"

show(gridplot([p1, p2], ncols=2))

### Performace - SYCL/CPU

In [16]:
data1 = json.load(open('results/roazhon13-1.rennes.grid5000.fr/results_sycl_cpu.json', 'r'))
labels =  [key for key, value in data1.items()]
i = 0

colors = ['green', 'red', 'blue', 'black', 'coral']
p1 = figure(title="Execution Times - SYCL/CPU Vs. MKL", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p1.xaxis.axis_label = 'Matrix size (N x N)'
p1.yaxis.axis_label = 'Time ( ms )'


for label in labels:
    times = list()
    errors = list()
    data_err = data1[label]
    sizes = [key for key, value in data_err.items()]
    for size in sizes:
        times.append(data_err[size]['time'])
        errors.append(data_err[size]['error'])
    sizes = list(map(int, sizes))

    data_plot = pd.DataFrame({'size':sizes, 'time':times, 'error':errors})
    # Add error bars to the DataFrame
    errors = list(map(float, errors))
    data_plot['error_low'] = data_plot['time'] - np.array(errors)
    data_plot['error_high'] = data_plot['time'] + np.array(errors)
    data_plot = data_plot.sort_values(by=['size'], ascending = True)
    data_plot = data_plot.reset_index(drop=True)
    source_sycl_roazhon13 = ColumnDataSource(data_plot)
    source_mkl_roazhon13 = ColumnDataSource(data_mkl_roazhon13)
    
    p1.square(source=source_sycl_roazhon13, x='size', y='time', legend_label=labels[i], fill_color=None, line_color='darkturquoise')
    p1.line(source=source_sycl_roazhon13, x='size', y='time', line_color='darkturquoise')
    
    p1.square(source=source_mkl_roazhon13, x='size', y='time', legend_label='mkl', fill_color=None, line_color='blue')
    p1.line(source=source_mkl_roazhon13, x='size', y='time', line_color='blue')
    
    i=i+1
p1.legend.location = "top_left"  

show(p1)

## Enery Consumption

In [17]:
# --------------------------------------------------------------------------------------------------------------
# Data
# --------------------------------------------------------------------------------------------------------------

mkl_roazhon13 = json.load(open('results/roazhon13-1.rennes.grid5000.fr/papi_mkl.json', 'r'))
sycl_cpu_roazhon13 = json.load(open('results/roazhon13-1.rennes.grid5000.fr/papi_sycl_cpu.json', 'r'))

data_mkl_roazhon13 = mkl_roazhon13['threads']['0']['regions']
data_sycl_cpu_roazhon13 = sycl_cpu_roazhon13['threads']['0']['regions']

# --------------------------------------------------------------------------------------------------------------
# Notes:
# - Los datos se tomaron 50 veces, de debe promediar
# - Los datos de GPU esta en miliWatts
# - Los datos en CPU esta en nanoJoules
# --------------------------------------------------------------------------------------------------------------
iterations = 50

# --------------------------------------------------------------------------------------------------------------
# MKL
# --------------------------------------------------------------------------------------------------------------
sizes = [key for key, value in data_mkl_roazhon13.items()]
counters = [key for key, value in data_mkl_roazhon13['0'].items()]
i = 0
energy_0_mkl_roazhon13 = list()
energy_1_mkl_roazhon13 = list()
dram_energy_0_mkl_roazhon13 = list()
dram_energy_1_mkl_roazhon13 = list()
times_mkl_roazhon13 = list()

for size in sizes:
    data_tmp = data_mkl_roazhon13[size]
    for counter in counters:     
        if counter == 'rapl:::PACKAGE_ENERGY:PACKAGE0':
            energy_0_mkl_roazhon13.append(int(data_tmp[counter])*1e-9)
        if counter == 'rapl:::PACKAGE_ENERGY:PACKAGE1':
            energy_1_mkl_roazhon13.append(int(data_tmp[counter])*1e-9)
        if counter == 'rapl:::DRAM_ENERGY:PACKAGE0':
            dram_energy_0_mkl_roazhon13.append(int(data_tmp[counter])*1e-9)
        if counter == 'rapl:::DRAM_ENERGY:PACKAGE1':
            dram_energy_1_mkl_roazhon13.append(int(data_tmp[counter])*1e-9)
        if counter == 'real_time_nsec':
            times_mkl_roazhon13.append(int(data_tmp[counter])*1e-9)
    i = i + 1
# --------------------------------------------------------------------------------------------------------------
energy_0_mkl_roazhon13 = [x / iterations for x in energy_0_mkl_roazhon13]
energy_1_mkl_roazhon13 = [x / iterations for x in energy_1_mkl_roazhon13]
dram_energy_0_mkl_roazhon13 = [x / iterations for x in dram_energy_0_mkl_roazhon13]
dram_energy_1_mkl_roazhon13 = [x / iterations for x in dram_energy_1_mkl_roazhon13]

mkl_energy_cpu_roazhon13 = [(x + y)/2 for (x, y) in zip(energy_0_mkl_roazhon13, energy_1_mkl_roazhon13)]
mkl_energy_dram_roazhon13 = [(x + y)/2 for (x, y) in zip(dram_energy_0_mkl_roazhon13, dram_energy_1_mkl_roazhon13)]
mkl_energy_roazhon13 = [x + y for (x, y) in zip(mkl_energy_cpu_roazhon13, mkl_energy_dram_roazhon13)] 

mkl_energy_w_roazhon13 = [x / y for (x, y) in zip(mkl_energy_roazhon13, times_mkl_roazhon13)] 



# --------------------------------------------------------------------------------------------------------------
# SYCL CPU
# --------------------------------------------------------------------------------------------------------------
sizes = [key for key, value in data_sycl_cpu_roazhon13.items()]
counters = [key for key, value in data_sycl_cpu_roazhon13['0'].items()]
i = 0
energy_0_sycl_cpu_roazhon13 = list()
energy_1_sycl_cpu_roazhon13 = list()
dram_energy_0_sycl_cpu_roazhon13 = list()
dram_energy_1_sycl_cpu_roazhon13 = list()
times_sycl_cpu_roazhon13 = list()

for size in sizes:
    data_tmp = data_sycl_cpu_roazhon13[size]
    for counter in counters:     
        if counter == 'rapl:::PACKAGE_ENERGY:PACKAGE0':
            energy_0_sycl_cpu_roazhon13.append(int(data_tmp[counter])*1e-9)
        if counter == 'rapl:::PACKAGE_ENERGY:PACKAGE1':
            energy_1_sycl_cpu_roazhon13.append(int(data_tmp[counter])*1e-9)
        if counter == 'rapl:::DRAM_ENERGY:PACKAGE0':
            dram_energy_0_sycl_cpu_roazhon13.append(int(data_tmp[counter])*1e-9)
        if counter == 'rapl:::DRAM_ENERGY:PACKAGE1':
            dram_energy_1_sycl_cpu_roazhon13.append(int(data_tmp[counter])*1e-9)
        if counter == 'real_time_nsec':
            times_sycl_cpu_roazhon13.append(int(data_tmp[counter])*1e-9)
    i = i + 1
# --------------------------------------------------------------------------------------------------------------
energy_0_sycl_cpu_roazhon13 = [x / iterations for x in energy_0_sycl_cpu_roazhon13]
energy_1_sycl_cpu_roazhon13 = [x / iterations for x in energy_1_sycl_cpu_roazhon13]
dram_energy_0_sycl_cpu_roazhon13 = [x / iterations for x in dram_energy_0_sycl_cpu_roazhon13]
dram_energy_1_sycl_cpu_roazhon13 = [x / iterations for x in dram_energy_1_sycl_cpu_roazhon13]

sycl_energy_cpu_roazhon13 = [(x + y)/2 for (x, y) in zip(energy_0_sycl_cpu_roazhon13, energy_1_sycl_cpu_roazhon13)]
sycl_energy_dram_roazhon13 = [(x + y)/2 for (x, y) in zip(dram_energy_0_sycl_cpu_roazhon13, dram_energy_1_sycl_cpu_roazhon13)]
sycl_energy_roazhon13 = [x + y for (x, y) in zip(sycl_energy_cpu_roazhon13, sycl_energy_dram_roazhon13)] 

sycl_energy_w_roazhon13 = [x / y for (x, y) in zip(sycl_energy_roazhon13, times_sycl_cpu_roazhon13)] 

In [18]:
# --------------------------------------------------------------------------------------------------------------
# Data source
# --------------------------------------------------------------------------------------------------------------
list_size = list([32, 48, 64, 80, 96, 112, 128, 256, 384, 512, 640, 768, 896, 1024, 2048, 3072, 4096, 5120, 6144, 7168, 8192])

# --------------------------------------------------------------------------------------------------------------
# MKL
# --------------------------------------------------------------------------------------------------------------
data_energy_mkl_roazhon13 = pd.DataFrame({'size':list_size,'PACKAGE_ENERGY0':energy_0_mkl_roazhon13, 'PACKAGE_ENERGY1':energy_1_mkl_roazhon13, 'DRAM_ENERGY0':dram_energy_0_mkl_roazhon13, 'DRAM_ENERGY1':dram_energy_1_mkl_roazhon13})
source_energy_mkl_roazhon13 = ColumnDataSource(data_energy_mkl_roazhon13)
# --------------------------------------------------------------------------------------------------------------
data_energy_mkl_total_roazhon13 = pd.DataFrame({'size':list_size, 'POWER':mkl_energy_roazhon13})
source_energy_mkl_total_roazhon13 = ColumnDataSource(data_energy_mkl_total_roazhon13)

# --------------------------------------------------------------------------------------------------------------
# SYCL CPU
# --------------------------------------------------------------------------------------------------------------
data_energy_sycl_cpu_roazhon13 = pd.DataFrame({'size':list_size,'PACKAGE_ENERGY0':energy_0_sycl_cpu_roazhon13, 'PACKAGE_ENERGY1':energy_1_sycl_cpu_roazhon13, 'DRAM_ENERGY0':dram_energy_0_sycl_cpu_roazhon13, 'DRAM_ENERGY1':dram_energy_1_sycl_cpu_roazhon13})
source_energy_sycl_cpu_roazhon13 = ColumnDataSource(data_energy_sycl_cpu_roazhon13)
# --------------------------------------------------------------------------------------------------------------
data_energy_sycl_total_roazhon13 = pd.DataFrame({'size':list_size, 'POWER':sycl_energy_roazhon13})
source_energy_sycl_total_roazhon13 = ColumnDataSource(data_energy_sycl_total_roazhon13)


In [19]:
# --------------------------------------------------------------------------------------------------------------
# Graph
# --------------------------------------------------------------------------------------------------------------
p1 = figure(title="Energy MKL", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p1.xaxis.axis_label = 'Matrix Size (N x N)'
p1.yaxis.axis_label = 'Energy (Joules)'

p2 = figure(title="Energy MKL (Average)", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p2.xaxis.axis_label = 'Matrix Size (N x N)'
p2.yaxis.axis_label = 'Energy (Joules)'

p7 = figure(title="Energy SYCL", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p7.xaxis.axis_label = 'Matrix Size (N x N)'
p7.yaxis.axis_label = 'Energy (Joules)'

p8 = figure(title="Energy SYCL (Average)", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p8.xaxis.axis_label = 'Matrix Size (N x N)'
p8.yaxis.axis_label = 'Energy (Joules)'

# --------------------------------------------------------------------------------------------------------------
# MKL
# --------------------------------------------------------------------------------------------------------------
p1.square(source=source_energy_mkl_roazhon13, x='size', y='PACKAGE_ENERGY0', legend_label='PACKAGE_ENERGY0_MKL', fill_color=None, line_color='orangered')
p1.line(source=source_energy_mkl_roazhon13, x='size', y='PACKAGE_ENERGY0', line_color='orangered')
p1.square(source=source_energy_mkl_roazhon13, x='size', y='PACKAGE_ENERGY1', legend_label='PACKAGE_ENERGY1_MKL', fill_color=None, line_color='orange')
p1.line(source=source_energy_mkl_roazhon13, x='size', y='PACKAGE_ENERGY1', line_color='orange')
p1.square(source=source_energy_mkl_roazhon13, x='size', y='DRAM_ENERGY0', legend_label='DRAM_ENERGY0_MKL', fill_color=None, line_color='mediumseagreen')
p1.line(source=source_energy_mkl_roazhon13, x='size', y='DRAM_ENERGY0', line_color='mediumseagreen')
p1.square(source=source_energy_mkl_roazhon13, x='size', y='DRAM_ENERGY1', legend_label='DRAM_ENERGY1_MKL', fill_color=None, line_color='teal')
p1.line(source=source_energy_mkl_roazhon13, x='size', y='DRAM_ENERGY1', line_color='teal')

p2.square(source=source_energy_mkl_total_roazhon13, x='size', y='POWER', legend_label='MKL - TOTAL POWER', fill_color=None, line_color='midnightblue')
p2.line(source=source_energy_mkl_total_roazhon13, x='size', y='POWER', line_color='midnightblue')

# --------------------------------------------------------------------------------------------------------------
# SYCL CPU
# --------------------------------------------------------------------------------------------------------------
p7.square(source=source_energy_sycl_cpu_roazhon13, x='size', y='PACKAGE_ENERGY0', legend_label='PACKAGE_ENERGY0_SYCL', fill_color=None, line_color='orangered')
p7.line(source=source_energy_sycl_cpu_roazhon13, x='size', y='PACKAGE_ENERGY0', line_color='orangered')
p7.square(source=source_energy_sycl_cpu_roazhon13, x='size', y='PACKAGE_ENERGY1', legend_label='PACKAGE_ENERGY1_SYCL', fill_color=None, line_color='orange')
p7.line(source=source_energy_sycl_cpu_roazhon13, x='size', y='PACKAGE_ENERGY1', line_color='orange')
p7.square(source=source_energy_sycl_cpu_roazhon13, x='size', y='DRAM_ENERGY0', legend_label='DRAM_ENERGY0_SYCL', fill_color=None, line_color='mediumseagreen')
p7.line(source=source_energy_sycl_cpu_roazhon13, x='size', y='DRAM_ENERGY0', line_color='mediumseagreen')
p7.square(source=source_energy_sycl_cpu_roazhon13, x='size', y='DRAM_ENERGY1', legend_label='DRAM_ENERGY1_SYCL', fill_color=None, line_color='teal')
p7.line(source=source_energy_sycl_cpu_roazhon13, x='size', y='DRAM_ENERGY1', line_color='teal')

p8.square(source=source_energy_sycl_total_roazhon13, x='size', y='POWER', legend_label='SYCL - TOTAL POWER', fill_color=None, line_color='darkturquoise')
p8.line(source=source_energy_sycl_total_roazhon13, x='size', y='POWER', line_color='darkturquoise')

p1.legend.location = "top_left"
p2.legend.location = "top_left"

p7.legend.location = "top_left"
p8.legend.location = "top_left"

show(gridplot([[p1, p2], [p7, p8]]))

In [20]:
p1 = figure(title="Energy Average", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'Matrix Size (N x N)'
p1.yaxis.axis_label = 'Energy (Joules)'

p1.square(source=source_energy_mkl_total_roazhon13, x='size', y='POWER', legend_label='MKL - TOTAL POWER', fill_color=None, line_color='midnightblue')
p1.line(source=source_energy_mkl_total_roazhon13, x='size', y='POWER', line_color='midnightblue')
p1.square(source=source_energy_sycl_total_roazhon13, x='size', y='POWER', legend_label='SYCL - TOTAL POWER', fill_color=None, line_color='darkturquoise')
p1.line(source=source_energy_sycl_total_roazhon13, x='size', y='POWER', line_color='darkturquoise')


p1.legend.location = "top_left"

show(p1)

## Power consumption

In [21]:
p1 = figure(title="Power Consumption", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'Matrix Size (N x N)'
p1.yaxis.axis_label = 'Power Consumption (Watts)'

data_energy_mkl_w_roazhon13 = pd.DataFrame({'size':list_size,'ENERGY':mkl_energy_w_roazhon13})
source_mkl_w_roazhon13 = ColumnDataSource(data_energy_mkl_w_roazhon13)

data_energy_sycl_w_roazhon13 = pd.DataFrame({'size':list_size,'ENERGY':sycl_energy_w_roazhon13})
source_sycl_w_roazhon13 = ColumnDataSource(data_energy_sycl_w_roazhon13)

p1.square(source=source_mkl_w_roazhon13, x='size', y='ENERGY', legend_label='mkl', color='midnightblue', fill_alpha=0.4)
p1.square(source=source_sycl_w_roazhon13, x='size', y='ENERGY', legend_label='SYCL', color='darkturquoise', fill_alpha=0.4)

p1.legend.location = "bottom_right"

show(p1)

## grouille-1.nancy.grid5000.fr
### Performace -CPU (AMD)

In [22]:
data = json.load(open('results/grouille-1.nancy.grid5000.fr/results_cpu.json', 'r'))
labels =  [key for key, value in data.items()]
i = 0

colors = ['green', 'red', 'blue', 'darkturquoise', 'coral']

p1 = figure(title="Execution Times - CPU", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'Matrix size (N x N)'
p1.yaxis.axis_label = 'Time ( ms )'

errors_total = list()

for label in labels:
    times = list()
    errors = list()
    data_err = data[label]
    sizes = [key for key, value in data_err.items()]
    for size in sizes:
        times.append(data_err[size]['time'])
        errors.append(data_err[size]['error'])
    sizes = list(map(int, sizes))

    data_plot = pd.DataFrame({'size':sizes, 'time':times, 'error':errors})
    # Add error bars to the DataFrame
    errors_total += [errors]
    errors = list(map(float, errors))
    errors_t = dict()
    errors_t['sizes'] = sizes
    for key, value in zip(labels, errors_total):
        errors_t[key] = value
    source_cpu_errors = ColumnDataSource(pd.DataFrame(errors_t))

    
    data_plot['error_low'] = data_plot['time'] - np.array(errors)
    data_plot['error_high'] = data_plot['time'] + np.array(errors)
    data_plot = data_plot.sort_values(by=['size'], ascending = True)
    data_plot = data_plot.reset_index(drop=True)
    source_cpu = ColumnDataSource(data_plot)
    
    p1.square(source=source_cpu, x='size', y='time', legend_label=labels[i], fill_color=None, line_color=colors[i])
    p1.line(source=source_cpu, x='size', y='time', line_color=colors[i])
        
    if label == 'openMP':
        data_openMP_grouille = data_plot.copy(deep = True)
        error_openMP_grouille = pd.DataFrame(errors_t)
        
    i=i+1

p1.legend.location = "top_left"

show(p1)

### Performace -GPU

In [23]:
data = json.load(open('results/grouille-1.nancy.grid5000.fr/results_gpu.json', 'r'))
labels =  [key for key, value in data.items()]
i = 0

colors = ['mediumseagreen', 'tomato', 'dodgerblue', 'deeppink', 'darkviolet']
p1 = figure(title="Execution Times - GPU", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p2 = figure(title="Error - GPU", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'Matrix size (N x N)'
p1.yaxis.axis_label = 'Time ( ms )'

p2.xaxis.axis_label = 'Matrix size (N x N)'
p2.yaxis.axis_label = 'MSE'

errors_total = list()

for label in labels:
    times = list()
    errors = list()
    data_err = data[label]
    sizes = [key for key, value in data_err.items()]
    for size in sizes:
        times.append(data_err[size]['time'])
        errors.append(data_err[size]['error'])
    sizes = list(map(int, sizes))

    data_plot = pd.DataFrame({'size':sizes, 'time':times, 'error':errors})
    # Add error bars to the DataFrame
    errors_total += [errors]
    errors = list(map(float, errors))
    errors_t = dict()
    errors_t['sizes'] = sizes
    for key, value in zip(labels, errors_total):
        errors_t[key] = value
    source_gpu_errors_grouille = ColumnDataSource(pd.DataFrame(errors_t))

    
    data_plot['error_low'] = data_plot['time'] - np.array(errors)
    data_plot['error_high'] = data_plot['time'] + np.array(errors)
    data_plot = data_plot.sort_values(by=['size'], ascending = True)
    data_plot = data_plot.reset_index(drop=True)
    source_gpu_grouille = ColumnDataSource(data_plot)
    
    p1.square(source=source_gpu_grouille, x='size', y='time', legend_label=labels[i], fill_color=None, line_color=colors[i])
    p1.line(source=source_gpu_grouille, x='size', y='time', line_color=colors[i])
    
    if label != 'serial':
        p2.square(source=source_gpu_errors_grouille, x='sizes', y=label, legend_label=labels[i], color=colors[i])
        #p2.line(source=source_gpu_errors, x='sizes', y=label, legend_label=labels[i], color=colors[i])
    
    if label == 'cuBLAS_Tensor':
        data_cublas_tensor_grouille = data_plot.copy(deep = True)
        error_cublas_tensor_grouille = pd.DataFrame(errors_t)
    if label == 'cuBLAS':
        data_cublas_grouille = data_plot.copy(deep = True)
        error_cublas_grouille = pd.DataFrame(errors_t)
        
    i=i+1

maxi = max(max([sublist[:] for sublist in errors_total]))


p2.y_range.end = maxi*1.1

p1.legend.location = "top_left"
p2.legend.location = "top_left"

show(gridplot([p1, p2], ncols=2))

In [24]:
p1 = figure(title="Execution Times - CPU", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p2 = figure(title="Error - CPU", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'Matrix size (N x N)'
p1.yaxis.axis_label = 'Time ( ms )'

p2.xaxis.axis_label = 'Matrix size (N x N)'
p2.yaxis.axis_label = 'MSE'


colors = ['green', 'red', 'blue', 'black', 'coral']

source_cublasT_grouille = ColumnDataSource(data_cublas_tensor_grouille)
source_cublasT_errors_grouille = ColumnDataSource(error_cublas_tensor_grouille)
source_cublas_grouille = ColumnDataSource(data_cublas_grouille)
source_cublas_errors_grouille = ColumnDataSource(error_cublas_grouille)

p1.square(source=source_cublasT_grouille, x='size', y='time', legend_label='cuBLAS_Tensor', fill_color=None, line_color='tomato')
p1.line(source=source_cublasT_grouille, x='size', y='time', line_color='tomato')
p1.square(source=source_cublas_grouille, x='size', y='time', legend_label='cuBLAS', fill_color=None, line_color='mediumseagreen')
p1.line(source=source_cublas_grouille, x='size', y='time', line_color='mediumseagreen')

p1.legend.location = "top_left"

p2.square(source=source_cublasT_errors_grouille, x='sizes', y='cuBLAS_Tensor', legend_label='cuBLAS_Tensor', color='tomato', fill_alpha=0.4)
#p2.line(source=source_cublasT_errors, x='sizes', y='cuBLAS_Tensor', line_color='tomato')
p2.square(source=source_cublas_errors_grouille, x='sizes', y='cuBLAS', legend_label='cuBLAS', color='mediumseagreen', fill_alpha=0.4)
#p2.line(source=source_cublas_errors, x='sizes', y='cuBLAS', line_color='mediumseagreen')
p2.y_range.end = maxi*1.1
p2.legend.location = "top_left"



show(gridplot([p1, p2], ncols=2))

### Performace - SYCL/CPU SYCL/GPU

In [60]:
data1 = json.load(open('results/grouille-1.nancy.grid5000.fr/results_sycl_cpu.json', 'r'))
labels =  [key for key, value in data1.items()]
i = 0

colors = ['green', 'red', 'blue', 'black', 'coral']
p1 = figure(title="Execution Times - SYCL/CPU Vs. openMP", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p1.xaxis.axis_label = 'Matrix size (N x N)'
p1.yaxis.axis_label = 'Time ( ms )'


for label in labels:
    times = list()
    errors = list()
    data_err = data1[label]
    sizes = [key for key, value in data_err.items()]
    for size in sizes:
        times.append(data_err[size]['time'])
        errors.append(data_err[size]['error'])
    sizes = list(map(int, sizes))

    data_plot = pd.DataFrame({'size':sizes, 'time':times, 'error':errors})
    # Add error bars to the DataFrame
    errors = list(map(float, errors))
    data_plot['error_low'] = data_plot['time'] - np.array(errors)
    data_plot['error_high'] = data_plot['time'] + np.array(errors)
    data_plot = data_plot.sort_values(by=['size'], ascending = True)
    data_plot = data_plot.reset_index(drop=True)
    source_sycl_grouille = ColumnDataSource(data_plot)
    source_openmp_grouille = ColumnDataSource(data_openMP_grouille)
    
    p1.square(source=source_sycl_grouille, x='size', y='time', legend_label=labels[i], fill_color=None, line_color='darkturquoise')
    p1.line(source=source_sycl_grouille, x='size', y='time', line_color='darkturquoise')
    
    p1.square(source=source_openmp_grouille, x='size', y='time', legend_label='openMP', fill_color=None, line_color='blue')
    p1.line(source=source_openmp_grouille, x='size', y='time', line_color='blue')
    
    i=i+1

#--------------------------------------------------------------------------------------------------------------------

data2 = json.load(open('results/grouille-1.nancy.grid5000.fr/results_sycl_gpu.json', 'r'))
labels =  [key for key, value in data2.items()]
i = 0

p2 = figure(title="Execution Times - SYCL/GPU Vs. cuBLAS", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p2.xaxis.axis_label = 'Matrix size (N x N)'
p2.yaxis.axis_label = 'Time ( ms )'

for label in labels:
    times = list()
    errors = list()
    data_err = data2[label]
    sizes = [key for key, value in data_err.items()]
    for size in sizes:
        times.append(data_err[size]['time'])
        errors.append(data_err[size]['error'])
    sizes = list(map(int, sizes))

    data_plot = pd.DataFrame({'size':sizes, 'time':times, 'error':errors})
    # Add error bars to the DataFrame
    errors = list(map(float, errors))
    data_plot['error_low'] = data_plot['time'] - np.array(errors)
    data_plot['error_high'] = data_plot['time'] + np.array(errors)
    data_plot = data_plot.sort_values(by=['size'], ascending = True)
    data_plot = data_plot.reset_index(drop=True)
    
    source_sycl_gpu_grouille = ColumnDataSource(data_plot)
    source_cublasT_grouille = ColumnDataSource(data_cublas_tensor_grouille)
    source_cublas_grouille = ColumnDataSource(data_cublas_grouille)
    
    p2.square(source=source_sycl_gpu_grouille, x='size', y='time', legend_label=labels[i], fill_color=None, line_color='mediumpurple')
    p2.line(source=source_sycl_gpu_grouille, x='size', y='time', line_color='mediumpurple')
    p2.segment(source=source_sycl_gpu_grouille, x0='size', y0='error_low', x1='size', y1='error_high', line_width=2)
    
    p2.square(source=source_cublasT_grouille, x='size', y='time', legend_label='cuBLAS_Tensor', fill_color=None, line_color='tomato')
    p2.line(source=source_cublasT_grouille, x='size', y='time', line_color='tomato')
    p2.segment(source=source_cublasT_grouille, x0='size', y0='error_low', x1='size', y1='error_high', line_width=2)
    
    p2.square(source=source_cublas_grouille, x='size', y='time', legend_label='cuBLAS', fill_color=None, line_color='mediumseagreen')
    p2.line(source=source_cublas_grouille, x='size', y='time', line_color='mediumseagreen')
    p2.segment(source=source_cublas_grouille, x0='size', y0='error_low', x1='size', y1='error_high', line_width=2)

    i=i+1
    
p1.legend.location = "top_left"  
p2.legend.location = "top_left"

show(gridplot([p1, p2], ncols=2))

## Enery Consumption

In [26]:
# --------------------------------------------------------------------------------------------------------------
# Data
# --------------------------------------------------------------------------------------------------------------

cublas_cpu = json.load(open('results/grouille-1.nancy.grid5000.fr/papi_cublas_cpu.json', 'r'))
cublas_gpu = json.load(open('results/grouille-1.nancy.grid5000.fr/papi_cublas_gpu.json', 'r'))

cublasT_cpu = json.load(open('results/grouille-1.nancy.grid5000.fr/papi_cublasT_cpu.json', 'r'))
cublasT_gpu = json.load(open('results/grouille-1.nancy.grid5000.fr/papi_cublasT_gpu.json', 'r'))

sycl_cpu = json.load(open('results/grouille-1.nancy.grid5000.fr/papi_sycl_cpu.json', 'r'))
sycl_gpu = json.load(open('results/grouille-1.nancy.grid5000.fr/papi_sycl_gpu.json', 'r'))
sycl_cpu_gpu = json.load(open('results/grouille-1.nancy.grid5000.fr/papi_sycl_cpu_gpu.json', 'r'))

data_cublas_cpu = cublas_cpu['threads']['0']['regions']
data_cublas_gpu = cublas_gpu['threads']['0']['regions']
data_cublasT_cpu = cublasT_cpu['threads']['0']['regions']
data_cublasT_gpu = cublasT_gpu['threads']['0']['regions']

data_sycl_cpu = sycl_cpu['threads']['0']['regions']

data_sycl_gpu = sycl_gpu['threads']['0']['regions']
data_sycl_cpu_gpu = sycl_cpu_gpu['threads']['0']['regions']

# --------------------------------------------------------------------------------------------------------------
# Notes:
# - Los datos se tomaron 50 veces, de debe promediar
# - Los datos de GPU esta en miliWatts
# - Los datos en CPU esta en nanoJoules
# --------------------------------------------------------------------------------------------------------------
iterations = 50

# --------------------------------------------------------------------------------------------------------------
# cuBLAS
# --------------------------------------------------------------------------------------------------------------
# CPU
# --------------------------------------------------------------------------------------------------------------
sizes = [key for key, value in data_cublas_cpu.items()]
counters = [key for key, value in data_cublas_cpu['0'].items()]
i = 0
energy_0_cublas_cpu_grouille = list()
energy_1_cublas_cpu_grouille = list()
times_cublas_grouille = list()

for size in sizes:
    data_tmp = data_cublas_cpu[size]
    for counter in counters:     
        if counter == 'rapl:::PACKAGE_ENERGY:PACKAGE0':
            energy_0_cublas_cpu_grouille.append(int(data_tmp[counter])*1e-9)
        if counter == 'rapl:::PACKAGE_ENERGY:PACKAGE1':
            energy_1_cublas_cpu_grouille.append(int(data_tmp[counter])*1e-9)
        if counter == 'real_time_nsec':
            times_cublas_grouille.append(int(data_tmp[counter])*1e-9)
    i = i + 1
# --------------------------------------------------------------------------------------------------------------
# GPU
# --------------------------------------------------------------------------------------------------------------   
sizes = [key for key, value in data_cublas_gpu.items()]
counters = [key for key, value in data_cublas_gpu['0'].items()]
i = 0
energy_cublas_gpu_grouille = list()
times_cublas_gpu_grouille = list()

for size in sizes:
    data_tmp = data_cublas_gpu[size]
    for counter in counters:     
        if counter == 'nvml:::NVIDIA_A100-PCIE-40GB:device_0:power':
            energy_cublas_gpu_grouille.append(int(data_tmp[counter])*1e-3)
        if counter == 'real_time_nsec':
            times_cublas_gpu_grouille.append(int(data_tmp[counter])*1e-9)
    i = i + 1
# --------------------------------------------------------------------------------------------------------------
energy_0_cublas_cpu_grouille = [x / iterations for x in energy_0_cublas_cpu_grouille]
energy_1_cublas_cpu_grouille = [x / iterations for x in energy_1_cublas_cpu_grouille]

cublas_energy_cpu_grouille = [(x + y)/2 for (x, y) in zip(energy_0_cublas_cpu_grouille, energy_1_cublas_cpu_grouille)]
cublas_energy_t_grouille = cublas_energy_cpu_grouille

energy_cublas_gpu_w_grouille = [x / iterations for x in energy_cublas_gpu_grouille]
energy_cublas_gpu_grouille = [x * y for (x, y) in zip(energy_cublas_gpu_w_grouille, times_cublas_gpu_grouille)]

cublas_energy_grouille = [x + y for (x, y) in zip(cublas_energy_t_grouille, energy_cublas_gpu_grouille)]


energy_0_cublas_w_grouille = [x / y for (x, y) in zip(energy_0_cublas_cpu_grouille, times_cublas_grouille)]
energy_1_cublas_w_grouille = [x / y for (x, y) in zip(energy_1_cublas_cpu_grouille, times_cublas_grouille)]

cublas_energy_w_grouille = [x / y for (x, y) in zip(cublas_energy_t_grouille, times_cublas_grouille)]
cublas_energy_w_grouille = [x + y for (x, y) in zip(cublas_energy_w_grouille, energy_cublas_gpu_w_grouille)] 

# --------------------------------------------------------------------------------------------------------------
# cuBLAS Tensor
# --------------------------------------------------------------------------------------------------------------
# CPU
# --------------------------------------------------------------------------------------------------------------
sizes = [key for key, value in data_cublasT_cpu.items()]
counters = [key for key, value in data_cublasT_cpu['0'].items()]
i = 0
energy_0_cublasT_cpu_grouille = list()
energy_1_cublasT_cpu_grouille = list()
times_cublasT_grouille = list()

for size in sizes:
    data_tmp = data_cublasT_cpu[size]
    for counter in counters:     
        if counter == 'rapl:::PACKAGE_ENERGY:PACKAGE0':
            energy_0_cublasT_cpu_grouille.append(int(data_tmp[counter])*1e-9)
        if counter == 'rapl:::PACKAGE_ENERGY:PACKAGE1':
            energy_1_cublasT_cpu_grouille.append(int(data_tmp[counter])*1e-9)
        if counter == 'real_time_nsec':
            times_cublasT_grouille.append(int(data_tmp[counter])*1e-9)
    i = i + 1

# --------------------------------------------------------------------------------------------------------------
# GPU
# --------------------------------------------------------------------------------------------------------------
sizes = [key for key, value in data_cublasT_gpu.items()]
counters = [key for key, value in data_cublasT_gpu['0'].items()]
i = 0
energy_cublasT_gpu_grouille = list()
times_cublasT_gpu_grouille = list()

for size in sizes:
    data_tmp = data_cublasT_gpu[size]
    for counter in counters:     
        if counter == 'nvml:::NVIDIA_A100-PCIE-40GB:device_0:power':
            energy_cublasT_gpu_grouille.append(int(data_tmp[counter])*1e-3)
        if counter == 'real_time_nsec':
            times_cublasT_gpu_grouille.append(int(data_tmp[counter])*1e-9)
    i = i + 1
# --------------------------------------------------------------------------------------------------------------
energy_0_cublasT_cpu_grouille = [x / iterations for x in energy_0_cublasT_cpu_grouille]
energy_1_cublasT_cpu_grouille = [x / iterations for x in energy_1_cublasT_cpu_grouille]

cublasT_energy_cpu_grouille = [(x + y)/2 for (x, y) in zip(energy_0_cublasT_cpu_grouille, energy_1_cublasT_cpu_grouille)]
cublasT_energy_t_grouille = cublasT_energy_cpu_grouille

energy_cublasT_gpu_w_grouille = [x / iterations for x in energy_cublasT_gpu_grouille]
energy_cublasT_gpu_grouille = [x * y for (x, y) in zip(energy_cublasT_gpu_w_grouille, times_cublasT_gpu_grouille)]

cublasT_energy_grouille = [x + y for (x, y) in zip(cublasT_energy_t_grouille, energy_cublasT_gpu_grouille)]

energy_0_cublasT_w_grouille = [x / y for (x, y) in zip(energy_0_cublasT_cpu_grouille, times_cublasT_grouille)]
energy_1_cublasT_w_grouille = [x / y for (x, y) in zip(energy_1_cublasT_cpu_grouille, times_cublasT_grouille)]

cublasT_energy_w_grouille = [x / y for (x, y) in zip(cublasT_energy_t_grouille, times_cublasT_grouille)]
cublasT_energy_w_grouille = [x + y for (x, y) in zip(cublasT_energy_w_grouille, energy_cublasT_gpu_w_grouille)]    
# --------------------------------------------------------------------------------------------------------------
# SYCL CPU
# --------------------------------------------------------------------------------------------------------------
sizes = [key for key, value in data_sycl_cpu.items()]
counters = [key for key, value in data_sycl_cpu['0'].items()]
i = 0
energy_0_sycl_cpu_grouille = list()
energy_1_sycl_cpu_grouille = list()
times_sycl_cpu_grouille = list()

for size in sizes:
    data_tmp = data_sycl_cpu[size]
    for counter in counters:     
        if counter == 'rapl:::PACKAGE_ENERGY:PACKAGE0':
            energy_0_sycl_cpu_grouille.append(int(data_tmp[counter])*1e-9)
        if counter == 'rapl:::PACKAGE_ENERGY:PACKAGE1':
            energy_1_sycl_cpu_grouille.append(int(data_tmp[counter])*1e-9)
        if counter == 'real_time_nsec':
            times_sycl_cpu_grouille.append(int(data_tmp[counter])*1e-9)
    i = i + 1
# --------------------------------------------------------------------------------------------------------------
energy_0_sycl_cpu_grouille = [x / iterations for x in energy_0_sycl_cpu_grouille]
energy_1_sycl_cpu_grouille = [x / iterations for x in energy_1_sycl_cpu_grouille]

sycl_energy_cpu_grouille = [(x + y)/2 for (x, y) in zip(energy_0_sycl_cpu_grouille, energy_1_sycl_cpu_grouille)]
sycl_energy_grouille = sycl_energy_cpu_grouille

sycl_energy_w_grouille = [x / y for (x, y) in zip(sycl_energy_grouille, times_sycl_cpu_grouille)] 

# --------------------------------------------------------------------------------------------------------------
# SYCL 
# --------------------------------------------------------------------------------------------------------------
# GPU
# --------------------------------------------------------------------------------------------------------------
sizes = [key for key, value in data_sycl_gpu.items()]
counters = [key for key, value in data_sycl_gpu['0'].items()]
i = 0
energy_sycl_gpu_grouille = list()
times_sycl_gpu_grouille = list()

for size in sizes:
    data_tmp = data_sycl_gpu[size]
    for counter in counters:     
        if counter == 'nvml:::NVIDIA_A100-PCIE-40GB:device_0:power':
            energy_sycl_gpu_grouille.append(int(data_tmp[counter])*1e-3)
        if counter == 'real_time_nsec':
            times_sycl_gpu_grouille.append(int(data_tmp[counter])*1e-9)
    i = i + 1

# --------------------------------------------------------------------------------------------------------------
# CPU
# --------------------------------------------------------------------------------------------------------------
sizes = [key for key, value in data_sycl_cpu_gpu.items()]
counters = [key for key, value in data_sycl_cpu_gpu['0'].items()]
i = 0
energy_0_sycl_cpu_gpu_grouille = list()
energy_1_sycl_cpu_gpu_grouille = list()
times_sycl_cpu_gpu_grouille = list()

for size in sizes:
    data_tmp = data_sycl_cpu_gpu[size]
    for counter in counters:     
        if counter == 'rapl:::PACKAGE_ENERGY:PACKAGE0':
            energy_0_sycl_cpu_gpu_grouille.append(int(data_tmp[counter])*1e-9)
        if counter == 'rapl:::PACKAGE_ENERGY:PACKAGE1':
            energy_1_sycl_cpu_gpu_grouille.append(int(data_tmp[counter])*1e-9)
        if counter == 'real_time_nsec':
            times_sycl_cpu_gpu_grouille.append(int(data_tmp[counter])*1e-9)
    i = i + 1
# --------------------------------------------------------------------------------------------------------------
energy_0_sycl_cpu_gpu_grouille = [x / iterations for x in energy_0_sycl_cpu_gpu_grouille]
energy_1_sycl_cpu_gpu_grouille = [x / iterations for x in energy_1_sycl_cpu_gpu_grouille]

sycl_gpu_energy_cpu_grouille = [(x + y)/2 for (x, y) in zip(energy_0_sycl_cpu_gpu_grouille, energy_1_sycl_cpu_gpu_grouille)]
sycl_gpu_energy_t_grouille = sycl_gpu_energy_cpu_grouille

energy_sycl_gpu_w_grouille = [x / iterations for x in energy_sycl_gpu_grouille]
energy_sycl_gpu_grouille = [x * y for (x, y) in zip(energy_sycl_gpu_w_grouille, times_sycl_gpu_grouille)]

sycl_gpu_energy_grouille = [x + y for (x, y) in zip(sycl_gpu_energy_t_grouille, energy_sycl_gpu_grouille)]

energy_0_sycl_w_grouille = [x / y for (x, y) in zip(energy_0_sycl_cpu_gpu_grouille, times_sycl_cpu_gpu_grouille)]
energy_1_sycl_w_grouille = [x / y for (x, y) in zip(energy_1_sycl_cpu_gpu_grouille, times_sycl_cpu_gpu_grouille)]

sycl_gpu_energy_w_grouille = [x / y for (x, y) in zip(sycl_gpu_energy_t_grouille, times_sycl_cpu_gpu_grouille)]
sycl_gpu_energy_w_grouille = [x + y for (x, y) in zip(sycl_gpu_energy_w_grouille, energy_sycl_gpu_w_grouille)]   

In [27]:
# --------------------------------------------------------------------------------------------------------------
# Data source
# --------------------------------------------------------------------------------------------------------------
list_size = list([32, 48, 64, 80, 96, 112, 128, 256, 384, 512, 640, 768, 896, 1024, 2048, 3072, 4096, 5120, 6144, 7168, 8192])

# --------------------------------------------------------------------------------------------------------------
# cuBLAS
# --------------------------------------------------------------------------------------------------------------
# CPU
# --------------------------------------------------------------------------------------------------------------
data_energy_cublas_cpu_grouille = pd.DataFrame({'size':list_size,'PACKAGE_ENERGY0':energy_0_cublas_cpu_grouille, 'PACKAGE_ENERGY1':energy_1_cublas_cpu_grouille})
source_energy_cublas_cpu_grouille = ColumnDataSource(data_energy_cublas_cpu_grouille)

# --------------------------------------------------------------------------------------------------------------
# GPU
# --------------------------------------------------------------------------------------------------------------   
data_energy_cublas_gpu_grouille = pd.DataFrame({'size':list_size,'POWER':energy_cublas_gpu_grouille})
source_energy_cublas_gpu_grouille = ColumnDataSource(data_energy_cublas_gpu_grouille)
# --------------------------------------------------------------------------------------------------------------   
data_energy_cublas_total_grouille = pd.DataFrame({'size':list_size, 'POWER':cublas_energy_grouille})
source_energy_cublas_total_grouille = ColumnDataSource(data_energy_cublas_total_grouille)


# --------------------------------------------------------------------------------------------------------------
# cuBLAS Tensor
# --------------------------------------------------------------------------------------------------------------
# CPU
# --------------------------------------------------------------------------------------------------------------
data_energy_cublasT_cpu_grouille = pd.DataFrame({'size':list_size,'PACKAGE_ENERGY0':energy_0_cublasT_cpu_grouille, 'PACKAGE_ENERGY1':energy_1_cublasT_cpu_grouille})
source_energy_cublasT_cpu_grouille = ColumnDataSource(data_energy_cublasT_cpu_grouille)
# --------------------------------------------------------------------------------------------------------------
# GPU
# --------------------------------------------------------------------------------------------------------------
data_energy_cublasT_gpu_grouille = pd.DataFrame({'size':list_size,'POWER':energy_cublasT_gpu_grouille})
source_energy_cublasT_gpu_grouille = ColumnDataSource(data_energy_cublasT_gpu_grouille)
# --------------------------------------------------------------------------------------------------------------   
data_energy_cublasT_total_grouille = pd.DataFrame({'size':list_size, 'POWER':cublasT_energy_grouille})
source_energy_cublasT_total_grouille = ColumnDataSource(data_energy_cublasT_total_grouille)

# --------------------------------------------------------------------------------------------------------------
# SYCL CPU
# --------------------------------------------------------------------------------------------------------------
data_energy_sycl_cpu_grouille = pd.DataFrame({'size':list_size,'PACKAGE_ENERGY0':energy_0_sycl_cpu_grouille, 'PACKAGE_ENERGY1':energy_1_sycl_cpu_grouille})
source_energy_sycl_cpu_grouille = ColumnDataSource(data_energy_sycl_cpu_grouille)
# --------------------------------------------------------------------------------------------------------------
data_energy_sycl_total_grouille = pd.DataFrame({'size':list_size, 'POWER':sycl_energy_grouille})
source_energy_sycl_total_grouille = ColumnDataSource(data_energy_sycl_total_grouille)

# --------------------------------------------------------------------------------------------------------------
# SYCL 
# --------------------------------------------------------------------------------------------------------------
# GPU
# --------------------------------------------------------------------------------------------------------------
data_energy_sycl_gpu_grouille = pd.DataFrame({'size':list_size,'POWER':energy_sycl_gpu_grouille})
source_energy_sycl_gpu_grouille = ColumnDataSource(data_energy_sycl_gpu_grouille)
# --------------------------------------------------------------------------------------------------------------
# CPU
# --------------------------------------------------------------------------------------------------------------
data_energy_sycl_cpu_gpu_grouille = pd.DataFrame({'size':list_size,'PACKAGE_ENERGY0':energy_0_sycl_cpu_gpu_grouille, 'PACKAGE_ENERGY1':energy_1_sycl_cpu_gpu_grouille})
source_energy_sycl_cpu_gpu_grouille = ColumnDataSource(data_energy_sycl_cpu_gpu_grouille)
# --------------------------------------------------------------------------------------------------------------   
data_energy_sycl_gpu_total_grouille = pd.DataFrame({'size':list_size, 'POWER':sycl_gpu_energy_grouille})
source_energy_sycl_gpu_total_grouille = ColumnDataSource(data_energy_sycl_gpu_total_grouille)

In [28]:
# --------------------------------------------------------------------------------------------------------------
# Graph
# --------------------------------------------------------------------------------------------------------------

p3 = figure(title="Energy cuBLAS", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p3.xaxis.axis_label = 'Matrix Size (N x N)'
p3.yaxis.axis_label = 'Energy ( Joules )'

p4 = figure(title="Energy cuBLAS (Average)", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p4.xaxis.axis_label = 'Matrix Size (N x N)'
p4.yaxis.axis_label = 'Energy ( Joules )'

p5 = figure(title="Energy cuBLAS Tensor", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p5.xaxis.axis_label = 'Matrix Size (N x N)'
p5.yaxis.axis_label = 'Energy ( Joules )'

p6 = figure(title="Energy cuBLAS Tensor (Average)", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p6.xaxis.axis_label = 'Matrix Size (N x N)'
p6.yaxis.axis_label = 'Energy ( Joules )'

p7 = figure(title="Energy SYCL", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p7.xaxis.axis_label = 'Matrix Size (N x N)'
p7.yaxis.axis_label = 'Energy ( Joules )'

p8 = figure(title="Energy SYCL (Average)", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p8.xaxis.axis_label = 'Matrix Size (N x N)'
p8.yaxis.axis_label = 'Energy ( Joules )'

p9 = figure(title="Energy SYCL/GPU", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p9.xaxis.axis_label = 'Matrix Size (N x N)'
p9.yaxis.axis_label = 'Energy ( Joules )'

p10 = figure(title="Energy SYCL/GPU (Average)", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p10.xaxis.axis_label = 'Matrix Size (N x N)'
p10.yaxis.axis_label = 'Energy ( Joules )'


# --------------------------------------------------------------------------------------------------------------
# cuBLAS
# --------------------------------------------------------------------------------------------------------------
p3.square(source=source_energy_cublas_cpu_grouille, x='size', y='PACKAGE_ENERGY0', legend_label='PACKAGE_ENERGY0_cuBLAS_cpu', fill_color=None, line_color='orangered')
p3.line(source=source_energy_cublas_cpu_grouille, x='size', y='PACKAGE_ENERGY0', line_color='orangered')
p3.square(source=source_energy_cublas_cpu_grouille, x='size', y='PACKAGE_ENERGY1', legend_label='PACKAGE_ENERGY1_cuBLAS_cpu', fill_color=None, line_color='orange')
p3.line(source=source_energy_cublas_cpu_grouille, x='size', y='PACKAGE_ENERGY1', line_color='orange')
p3.square(source=source_energy_cublas_gpu_grouille, x='size', y='POWER', legend_label='GPU cuBLAS', fill_color=None, line_color='brown')
p3.line(source=source_energy_cublas_gpu_grouille, x='size', y='POWER', line_color='brown')

p4.square(source=source_energy_cublas_total_grouille, x='size', y='POWER', legend_label='cuBLAS - TOTAL POWER', fill_color=None, line_color='mediumseagreen')
p4.line(source=source_energy_cublas_total_grouille, x='size', y='POWER', line_color='mediumseagreen')

# --------------------------------------------------------------------------------------------------------------
# cuBLAS Tensor
# --------------------------------------------------------------------------------------------------------------
p5.square(source=source_energy_cublasT_cpu_grouille, x='size', y='PACKAGE_ENERGY0', legend_label='PACKAGE_ENERGY0_cuBLAS_Tensor_cpu', fill_color=None, line_color='orangered')
p5.line(source=source_energy_cublasT_cpu_grouille, x='size', y='PACKAGE_ENERGY0', line_color='orangered')
p5.square(source=source_energy_cublasT_cpu_grouille, x='size', y='PACKAGE_ENERGY1', legend_label='PACKAGE_ENERGY1_cuBLAS_Tensor_cpu', fill_color=None, line_color='orange')
p5.line(source=source_energy_cublasT_cpu_grouille, x='size', y='PACKAGE_ENERGY1', line_color='orange')
p5.square(source=source_energy_cublasT_gpu_grouille, x='size', y='POWER', legend_label='GPU cuBLAS Tensor', fill_color=None, line_color='brown')
p5.line(source=source_energy_cublasT_gpu_grouille, x='size', y='POWER', line_color='brown')

p6.square(source=source_energy_cublasT_total_grouille, x='size', y='POWER', legend_label='cuBLAS Tensor - TOTAL POWER', fill_color=None, line_color='tomato')
p6.line(source=source_energy_cublasT_total_grouille, x='size', y='POWER', line_color='tomato')

# --------------------------------------------------------------------------------------------------------------
# SYCL CPU
# --------------------------------------------------------------------------------------------------------------
p7.square(source=source_energy_sycl_cpu_grouille, x='size', y='PACKAGE_ENERGY0', legend_label='PACKAGE_ENERGY0_SYCL', fill_color=None, line_color='orangered')
p7.line(source=source_energy_sycl_cpu_grouille, x='size', y='PACKAGE_ENERGY0', line_color='orangered')
p7.square(source=source_energy_sycl_cpu_grouille, x='size', y='PACKAGE_ENERGY1', legend_label='PACKAGE_ENERGY1_SYCL', fill_color=None, line_color='orange')
p7.line(source=source_energy_sycl_cpu_grouille, x='size', y='PACKAGE_ENERGY1', line_color='orange')

p8.square(source=source_energy_sycl_total_grouille, x='size', y='POWER', legend_label='SYCL - TOTAL POWER', fill_color=None, line_color='darkturquoise')
p8.line(source=source_energy_sycl_total_grouille, x='size', y='POWER', line_color='darkturquoise')

# --------------------------------------------------------------------------------------------------------------
# SYCL GPU
# --------------------------------------------------------------------------------------------------------------
p9.square(source=source_energy_sycl_cpu_gpu_grouille, x='size', y='PACKAGE_ENERGY0', legend_label='PACKAGE_ENERGY0_SYCL_GPU', fill_color=None, line_color='orangered')
p9.line(source=source_energy_sycl_cpu_gpu_grouille, x='size', y='PACKAGE_ENERGY0', line_color='orangered')
p9.square(source=source_energy_sycl_cpu_gpu_grouille, x='size', y='PACKAGE_ENERGY1', legend_label='PACKAGE_ENERGY1_SYCL_GPU', fill_color=None, line_color='orange')
p9.line(source=source_energy_sycl_cpu_gpu_grouille, x='size', y='PACKAGE_ENERGY1', line_color='orange')
p9.square(source=source_energy_sycl_gpu_grouille, x='size', y='POWER', legend_label='GPU SYCL_GPU', fill_color=None, line_color='brown')
p9.line(source=source_energy_sycl_gpu_grouille, x='size', y='POWER', line_color='brown')

p10.square(source=source_energy_sycl_gpu_total_grouille, x='size', y='POWER', legend_label='SYCL_GPU - TOTAL POWER', fill_color=None, line_color='mediumpurple')
p10.line(source=source_energy_sycl_gpu_total_grouille, x='size', y='POWER', line_color='mediumpurple')
# --------------------------------------------------------------------------------------------------------------

p3.legend.location = "top_left"
p4.legend.location = "top_left"
p5.legend.location = "top_left"
p6.legend.location = "top_left"
p7.legend.location = "top_left"
p8.legend.location = "top_left"
p9.legend.location = "top_left"
p10.legend.location = "top_left"

show(gridplot([[p3,p4], [p5,p6], [p7, p8], [p9, p10]]))

In [29]:
p1 = figure(title="Energy Average", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p2 = figure(title="Energy Average", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'Matrix Size (N x N)'
p1.yaxis.axis_label = 'Energy ( Joules )'
p2.xaxis.axis_label = 'Matrix Size (N x N)'
p2.yaxis.axis_label = 'Energy ( Joules )'

p1.square(source=source_energy_cublas_total_grouille, x='size', y='POWER', legend_label='cuBLAS - TOTAL POWER', fill_color=None, line_color='mediumseagreen')
p1.line(source=source_energy_cublas_total_grouille, x='size', y='POWER', line_color='mediumseagreen')
p1.square(source=source_energy_cublasT_total_grouille, x='size', y='POWER', legend_label='cuBLAS Tensor - TOTAL POWER', fill_color=None, line_color='tomato')
p1.line(source=source_energy_cublasT_total_grouille, x='size', y='POWER', line_color='tomato')
p1.square(source=source_energy_sycl_total_grouille, x='size', y='POWER', legend_label='SYCL - TOTAL POWER', fill_color=None, line_color='darkturquoise')
p1.line(source=source_energy_sycl_total_grouille, x='size', y='POWER', line_color='darkturquoise')
p1.square(source=source_energy_sycl_gpu_total_grouille, x='size', y='POWER', legend_label='SYCL_GPU - TOTAL POWER', fill_color=None, line_color='mediumpurple')
p1.line(source=source_energy_sycl_gpu_total_grouille, x='size', y='POWER', line_color='mediumpurple')

p2.square(source=source_energy_cublasT_total_grouille, x='size', y='POWER', legend_label='cuBLAS Tensor - TOTAL POWER', fill_color=None, line_color='tomato')
p2.line(source=source_energy_cublasT_total_grouille, x='size', y='POWER', line_color='tomato')

p1.legend.location = "top_left"
p2.legend.location = "top_left"

show(gridplot([p1, p2], ncols=2))

## Power consumption

In [30]:
p1 = figure(title="Power Consumption", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p2 = figure(title="Power Consumption", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'Matrix Size (N x N)'
p1.yaxis.axis_label = 'Power Consumption ( Watts )'
p2.xaxis.axis_label = 'Matrix Size (N x N)'
p2.yaxis.axis_label = 'Power Consumption ( Watts )'


data_energy_cublas_w_grouille = pd.DataFrame({'size':list_size,'ENERGY':cublas_energy_w_grouille})
source_cublas_w_grouille = ColumnDataSource(data_energy_cublas_w_grouille)

data_energy_cublasT_w_grouille = pd.DataFrame({'size':list_size,'ENERGY':cublasT_energy_w_grouille})
source_cublasT_w_grouille = ColumnDataSource(data_energy_cublasT_w_grouille)

data_energy_sycl_w_grouille = pd.DataFrame({'size':list_size,'ENERGY':sycl_energy_w_grouille})
source_sycl_w_grouille = ColumnDataSource(data_energy_sycl_w_grouille)

data_energy_sycl_gpu_w_grouille = pd.DataFrame({'size':list_size,'ENERGY':sycl_gpu_energy_w_grouille})
source_sycl_gpu_w_grouille = ColumnDataSource(data_energy_sycl_gpu_w_grouille)

p1.square(source=source_cublas_w_grouille, x='size', y='ENERGY', legend_label='cuBLAS', color='mediumseagreen', fill_alpha=0.4)
p1.square(source=source_cublasT_w_grouille, x='size', y='ENERGY', legend_label='cuBLAS Tensor', color='tomato', fill_alpha=0.4)
p1.square(source=source_sycl_w_grouille, x='size', y='ENERGY', legend_label='SYCL', color='darkturquoise', fill_alpha=0.4)
p1.square(source=source_sycl_gpu_w_grouille, x='size', y='ENERGY', legend_label='SYCL GPU', color='mediumpurple', fill_alpha=0.4)

p1.legend.location = "top_left"

show(p1)

### Final

In [31]:
p1 = figure(title="Energy Consumption", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p2 = figure(title="Error", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p3 = figure(title="Energy Consumption", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p4 = figure(title="Power Consumption", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")


p1.xaxis.axis_label = 'Matrix Size (N x N)'
p1.yaxis.axis_label = 'Execution Times ( ms )'
p2.xaxis.axis_label = 'Matrix Size (N x N)'
p2.yaxis.axis_label = 'MSE'
p3.xaxis.axis_label = 'Matrix Size (N x N)'
p3.yaxis.axis_label = 'Energy ( Joules )'
p4.xaxis.axis_label = 'Matrix Size (N x N)'
p4.yaxis.axis_label = 'Power Consumption ( Watts )'

p1.square(source=source_cublasT_grouille, x='size', y='time', legend_label='cuBLAS_Tensor', fill_color=None, line_color='tomato')
p1.line(source=source_cublasT_grouille, x='size', y='time', line_color='tomato')
#p1.square(source=source_mkl, x='size', y='time', legend_label='mkl', fill_color=None, line_color='blue')
#p1.line(source=source_mkl, x='size', y='time', line_color='blue')

p2.square(source=source_cublasT_errors_grouille, x='sizes', y='cuBLAS_Tensor', legend_label='cuBLAS_Tensor', color='tomato', fill_alpha=0.4)
#p2.line(source=source_cublasT_errors, x='sizes', y='cuBLAS_Tensor', line_color='tomato')
#p2.square(source=source_mkl_errors, x='sizes', y='mkl', legend_label='mkl', color='blue', fill_alpha=0.4)
#p2.line(source=source_mkl_errors, x='sizes', y='mkl', line_color='blue')

#p3.square(source=source_energy_mkl_total, x='size', y='POWER', legend_label='MKL - TOTAL POWER', fill_color=None, line_color='midnightblue')
#p3.line(source=source_energy_mkl_total, x='size', y='POWER', line_color='midnightblue')
p3.square(source=source_energy_cublasT_total_grouille, x='size', y='POWER', legend_label='cuBLAS Tensor - TOTAL POWER', fill_color=None, line_color='tomato')
p3.line(source=source_energy_cublasT_total_grouille, x='size', y='POWER', line_color='tomato')

#p4.square(source=source_mkl_w, x='size', y='ENERGY', legend_label='mkl', color='midnightblue', fill_alpha=0.4)
p4.square(source=source_cublasT_w_grouille, x='size', y='ENERGY', legend_label='cuBLAS Tensor', color='tomato', fill_alpha=0.4)

p1.legend.location = "top_left"
p2.legend.location = "top_left"
p3.legend.location = "top_left"
p4.legend.location = "top_left"

show(gridplot([p1, p2, p3, p4], ncols=4))

_____________

## Resultados Comparativos

### CPU

In [32]:
source_mkl_chifflot
source_mkl_errors_chifflot
source_mkl_roazhon13
source_mkl_errors_roazhon13

ColumnDataSource(id='p4481', ...)

In [33]:
p1 = figure(title="Execution Times - CPU", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p2 = figure(title="Error - CPU", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'Matrix size (N x N)'
p1.yaxis.axis_label = 'Time ( ms )'
p2.xaxis.axis_label = 'Matrix size (N x N)'
p2.yaxis.axis_label = 'MSE'


p1.square(source=source_mkl_chifflot, x='size', y='time', legend_label='mkl_chifflot', fill_color=None, line_color='blue')
p1.line(source=source_mkl_chifflot, x='size', y='time', line_color='blue')
p1.square(source=source_mkl_roazhon13, x='size', y='time', legend_label='mkl_roazhon13', fill_color=None, line_color='darkturquoise')
p1.line(source=source_mkl_roazhon13, x='size', y='time', line_color='darkturquoise')

p1.legend.location = "top_left"

p2.square(source=source_mkl_errors_chifflot, x='sizes', y='mkl', legend_label='mkl_chifflot', color='blue', fill_alpha=0.4)
p2.square(source=source_mkl_errors_roazhon13, x='sizes', y='mkl', legend_label='mkl_roazhon13', color='darkturquoise', fill_alpha=0.4)
p2.legend.location = "top_left"

show(gridplot([p1, p2], ncols=2))

In [34]:
source_openmp_chifflot
source_openmp_roazhon13
source_openmp_grouille

ColumnDataSource(id='p5747', ...)

In [35]:
p1 = figure(title="Execution Times - CPU", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'Matrix size (N x N)'
p1.yaxis.axis_label = 'Time ( ms )'

p1.square(source=source_openmp_chifflot, x='size', y='time', legend_label='openmp_chifflot', fill_color=None, line_color='blue')
p1.line(source=source_openmp_chifflot, x='size', y='time', line_color='blue')
p1.square(source=source_openmp_roazhon13, x='size', y='time', legend_label='openmp_roazhon13', fill_color=None, line_color='red')
p1.line(source=source_openmp_roazhon13, x='size', y='time', line_color='red')
p1.square(source=source_openmp_grouille, x='size', y='time', legend_label='openmp_grouille', fill_color=None, line_color='orange')
p1.line(source=source_openmp_grouille, x='size', y='time', line_color='orange')

p1.legend.location = "top_left"

show(p1)

### GPU

In [36]:
source_cublasT_chifflot
source_cublasT_errors_chifflot
source_cublasT_grouille
source_cublasT_errors_grouille

ColumnDataSource(id='p5617', ...)

In [37]:
p1 = figure(title="Execution Times - GPU", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p2 = figure(title="Error - GPU", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'Matrix size (N x N)'
p1.yaxis.axis_label = 'Time ( ms )'

p2.xaxis.axis_label = 'Matrix size (N x N)'
p2.yaxis.axis_label = 'MSE'

colors = ['green', 'red', 'blue', 'black', 'coral']

p1.square(source=source_cublasT_chifflot, x='size', y='time', legend_label='cuBLAS_Tensor_chifflot', fill_color=None, line_color='tomato')
p1.line(source=source_cublasT_chifflot, x='size', y='time', line_color='tomato')
p1.square(source=source_cublasT_grouille, x='size', y='time', legend_label='cuBLAS_Tensor_grouille', fill_color=None, line_color='mediumseagreen')
p1.line(source=source_cublasT_grouille, x='size', y='time', line_color='mediumseagreen')

p1.legend.location = "top_left"

p2.square(source=source_cublasT_errors_chifflot, x='sizes', y='cuBLAS_Tensor', legend_label='cuBLAS_Tensor_chifflot', color='tomato', fill_alpha=0.4)
p2.square(source=source_cublasT_errors_grouille, x='sizes', y='cuBLAS_Tensor', legend_label='cuBLAS_Tensor_grouille', color='mediumseagreen', fill_alpha=0.4)
p2.legend.location = "top_left"

show(gridplot([p1, p2], ncols=2))

In [38]:
source_cublas_chifflot
source_cublas_errors_chifflot
source_cublas_grouille
source_cublas_errors_grouille

ColumnDataSource(id='p5623', ...)

In [39]:
p1 = figure(title="Execution Times - GPU", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p2 = figure(title="Error - GPU", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'Matrix size (N x N)'
p1.yaxis.axis_label = 'Time ( ms )'

p2.xaxis.axis_label = 'Matrix size (N x N)'
p2.yaxis.axis_label = 'MSE'

colors = ['green', 'red', 'blue', 'black', 'coral']

p1.square(source=source_cublas_chifflot, x='size', y='time', legend_label='cuBLAS_chifflot', fill_color=None, line_color='tomato')
p1.line(source=source_cublas_chifflot, x='size', y='time', line_color='tomato')
p1.square(source=source_cublas_grouille, x='size', y='time', legend_label='cuBLAS_grouille', fill_color=None, line_color='mediumseagreen')
p1.line(source=source_cublas_grouille, x='size', y='time', line_color='mediumseagreen')

p1.legend.location = "top_left"

p2.square(source=source_cublas_errors_chifflot, x='sizes', y='cuBLAS', legend_label='cuBLAS_chifflot', color='tomato', fill_alpha=0.4)
p2.square(source=source_cublas_errors_grouille, x='sizes', y='cuBLAS', legend_label='cuBLAS_grouille', color='mediumseagreen', fill_alpha=0.4)
p2.legend.location = "top_left"

show(gridplot([p1, p2], ncols=2))

### SYCL

In [40]:
source_sycl_chifflot
source_sycl_roazhon13
source_sycl_grouille

ColumnDataSource(id='p5744', ...)

In [41]:
p1 = figure(title="Execution Times - CPU", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'Matrix size (N x N)'
p1.yaxis.axis_label = 'Time ( ms )'

p1.square(source=source_sycl_chifflot, x='size', y='time', legend_label="sycl_chifflot", fill_color=None, line_color='mediumpurple')
p1.line(source=source_sycl_chifflot, x='size', y='time', line_color='mediumpurple')
p1.square(source=source_sycl_roazhon13, x='size', y='time', legend_label='sycl_roazhon13', fill_color=None, line_color='red')
p1.line(source=source_sycl_roazhon13, x='size', y='time', line_color='red')
p1.square(source=source_sycl_grouille, x='size', y='time', legend_label='sycl_grouille', fill_color=None, line_color='orange')
p1.line(source=source_sycl_grouille, x='size', y='time', line_color='orange')

p1.legend.location = "top_left"

show(p1)


In [42]:
source_sycl_gpu_chifflot
source_sycl_gpu_grouille

ColumnDataSource(id='p5819', ...)

In [43]:
p1 = figure(title="Execution Times - GPU", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'Matrix size (N x N)'
p1.yaxis.axis_label = 'Time ( ms )'

p1.square(source=source_sycl_gpu_chifflot, x='size', y='time', legend_label="sycl_gpu_chifflot", fill_color=None, line_color='mediumpurple')
p1.line(source=source_sycl_gpu_chifflot, x='size', y='time', line_color='mediumpurple')
p1.square(source=source_sycl_gpu_grouille, x='size', y='time', legend_label='sycl_gpu_grouille', fill_color=None, line_color='red')
p1.line(source=source_sycl_gpu_grouille, x='size', y='time', line_color='red')

p1.legend.location = "top_left"

show(p1)

### ENERGY

In [44]:
source_energy_mkl_total_papi_chifflot
source_energy_mkl_total_roazhon13

ColumnDataSource(id='p4672', ...)

In [45]:
p1 = figure(title="Energy Consumption", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p1.xaxis.axis_label = 'Matrix Size (N x N)'
p1.yaxis.axis_label = 'Energy ( Joules )'
p1.square(source=source_energy_mkl_total_papi_chifflot, x='size', y='POWER', legend_label='mkl_chifflot', color='tomato', fill_alpha=0.4)
p1.square(source=source_energy_mkl_total_roazhon13, x='size', y='POWER', legend_label='mkl_roazhon13', color='blue', fill_alpha=0.4)
show(p1)

In [46]:
source_energy_cublas_total_papi_chifflot
source_energy_cublas_total_grouille

source_energy_cublasT_total_papi_chifflot
source_energy_cublasT_total_grouille

ColumnDataSource(id='p5967', ...)

In [47]:
p1 = figure(title="Energy Consumption", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p1.xaxis.axis_label = 'Matrix Size (N x N)'
p1.yaxis.axis_label = 'Energy ( Joules )'
p1.square(source=source_energy_cublas_total_papi_chifflot, x='size', y='POWER', legend_label='cuBLAS chifflot', color='tomato', fill_alpha=0.4)
p1.square(source=source_energy_cublas_total_grouille, x='size', y='POWER', legend_label='cuBLAS roazhon13', color='blue', fill_alpha=0.4)
p1.square(source=source_energy_cublasT_total_papi_chifflot, x='size', y='POWER', legend_label='cuBLAS Tensor chifflot', color='green', fill_alpha=0.4)
p1.square(source=source_energy_cublasT_total_grouille, x='size', y='POWER', legend_label='cuBLAS Tensor roazhon13', color='orange', fill_alpha=0.4)


show(p1)

In [48]:
source_energy_sycl_total_papi_chifflot
source_energy_sycl_total_roazhon13
source_energy_sycl_total_grouille

ColumnDataSource(id='p5973', ...)

In [49]:
p1 = figure(title="Energy Consumption", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p1.xaxis.axis_label = 'Matrix Size (N x N)'
p1.yaxis.axis_label = 'Energy ( Joules )'
p1.square(source=source_energy_sycl_total_papi_chifflot, x='size', y='POWER', legend_label='sycl chifflot', color='tomato', fill_alpha=0.4)
p1.square(source=source_energy_sycl_total_roazhon13, x='size', y='POWER', legend_label='sycl roazhon13', color='blue', fill_alpha=0.4)
p1.square(source=source_energy_sycl_total_grouille, x='size', y='POWER', legend_label='sycl grouille', color='green', fill_alpha=0.4)

show(p1)

In [50]:
source_energy_sycl_gpu_total_papi_chifflot
source_energy_sycl_gpu_total_grouille

ColumnDataSource(id='p5982', ...)

In [51]:
p1 = figure(title="Energy Consumption", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p1.xaxis.axis_label = 'Matrix Size (N x N)'
p1.yaxis.axis_label = 'Energy ( Joules )'
p1.square(source=source_energy_sycl_gpu_total_papi_chifflot, x='size', y='POWER', legend_label='sycl GPU chifflot', color='tomato', fill_alpha=0.4)
p1.square(source=source_energy_sycl_gpu_total_grouille, x='size', y='POWER', legend_label='sycl GPU grouille', color='blue', fill_alpha=0.4)

show(p1)

### POWER

In [52]:
source_mkl_w_papi_chifflot
source_mkl_w_roazhon13

ColumnDataSource(id='p5127', ...)

In [53]:
p1 = figure(title="Power Consumption", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'Matrix Size (N x N)'
p1.yaxis.axis_label = 'Power Consumption ( Watts )'

p1.square(source=source_mkl_w_papi_chifflot, x='size', y='ENERGY', legend_label='mkl chifflot', color='mediumseagreen', fill_alpha=0.4)
p1.square(source=source_mkl_w_roazhon13, x='size', y='ENERGY', legend_label='mkl roazhon13', color='tomato', fill_alpha=0.4)

p1.legend.location = "top_left"

show(p1)

In [54]:
source_cublas_w_papi_chifflot
source_cublas_w_grouille

source_cublasT_w_papi_chifflot
source_cublasT_w_grouille

ColumnDataSource(id='p6805', ...)

In [55]:
p1 = figure(title="Power Consumption", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'Matrix Size (N x N)'
p1.yaxis.axis_label = 'Power Consumption ( Watts )'

p1.square(source=source_cublas_w_papi_chifflot, x='size', y='ENERGY', legend_label='cuBLAS chifflot', color='mediumseagreen', fill_alpha=0.4)
p1.square(source=source_cublas_w_grouille, x='size', y='ENERGY', legend_label='cuBLAS grouille', color='tomato', fill_alpha=0.4)
p1.square(source=source_cublasT_w_papi_chifflot, x='size', y='ENERGY', legend_label='cuBLAS Tensor chifflot', color='blue', fill_alpha=0.4)
p1.square(source=source_cublasT_w_grouille, x='size', y='ENERGY', legend_label='cuBLAS Tensor grouille', color='orange', fill_alpha=0.4)

p1.legend.location = "top_left"

show(p1)

In [56]:
source_sycl_w_papi_chifflot
source_sycl_w_roazhon13
source_sycl_w_grouille

ColumnDataSource(id='p6808', ...)

In [57]:
p1 = figure(title="Power Consumption", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'Matrix Size (N x N)'
p1.yaxis.axis_label = 'Power Consumption ( Watts )'

p1.square(source=source_sycl_w_papi_chifflot, x='size', y='ENERGY', legend_label='sycl chifflot', color='mediumseagreen', fill_alpha=0.4)
p1.square(source=source_sycl_w_roazhon13, x='size', y='ENERGY', legend_label='sycl roazhon13', color='tomato', fill_alpha=0.4)
p1.square(source=source_sycl_w_grouille, x='size', y='ENERGY', legend_label='sycl grouille', color='blue', fill_alpha=0.4)

p1.legend.location = "top_left"

show(p1)

In [58]:
source_sycl_gpu_w_papi_chifflot
source_sycl_gpu_w_grouille

ColumnDataSource(id='p6811', ...)

In [59]:
p1 = figure(title="Power Consumption", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'Matrix Size (N x N)'
p1.yaxis.axis_label = 'Power Consumption ( Watts )'

p1.square(source=source_sycl_gpu_w_papi_chifflot, x='size', y='ENERGY', legend_label='sycl GPU chifflot', color='mediumseagreen', fill_alpha=0.4)
p1.square(source=source_sycl_gpu_w_grouille, x='size', y='ENERGY', legend_label='sycl GPU grouille', color='blue', fill_alpha=0.4)

p1.legend.location = "top_left"

show(p1)